<a href="https://colab.research.google.com/github/justsvykas/Is-Chaos-Good-For-Reservoir-Computing-/blob/main/Code_for_Honours_thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduciton

This code is written for results in thesis **Is Chaos Good for Reservoir Computing?** which can be found [here](https://github.com/justsvykas/Is-Chaos-Good-For-Reservoir-Computing-/blob/main/Thesis.pdf).



# Table of Contents

- **Reservoir**: This section defines customizable neural network functions
- **Analysis Functions**: This section defines all functions which analyses neural networks
- **Reservoir analysis**: This is result section which analyses various reservoirs
- **Looped Network**: This is result section which analyses looped networks
- **Bifurcations**: this is result section which analyses networks using bifurcation diagrams
- **Individual neurons investigation**: This is result section which analyses specific neurons of some networks
- **Orbits**: This is result section which analyses neural networks using orbit diagrams


# Reservoir


## Chaotic Map

In [ ]:
def F_mod(s_n_minus_1):
    return (0.2 + -1.1 * s_n_minus_1) % 1

In [ ]:
def F_identity(s_n_minus_1):
  return s_n_minus_1

## Weight Matrix

In [ ]:
import numpy as np
import scipy.sparse.linalg as sp
import time
from scipy.sparse.linalg import ArpackNoConvergence


def initialize_reservoir(N, epsilon, rho, ex_time=False):
    """
    Initialise the reservoir layer autonomously.

    Parameters:
    - N: Number of reservoir units.
    - epsilon: Sparsity factor for the matrix.
    - rho: Spectral radius to be rescaled to.

    Returns:
    - W: Initialized reservoir matrix.
    """
    # We set seed so results can be reproducable
    np.random.seed(157)

    # Start the timer
    start_time = time.time()

    # Making NxN matrix with random values from -1 to 1
    W = -1+ (2*1) * np.random.rand(N, N)

    # Making 1-epsilon proportion of matrix entries zeroes
    for i in range(np.size(W)):
        if np.random.rand() > epsilon:
            W.flat[i] = 0

    try:
        # Finding the largest eigenvalue
        largest = np.abs(sp.eigs(W, k=1, which='LM', maxiter=5000)[0][0])  # 'LM' means largest magnitude
    except ArpackNoConvergence:
        largest = np.max(np.abs(np.linalg.eigvals(W)))

    # Rescale such that spectral radius is rho
    W = (W / largest) * rho

    # Calculate execution time
    if ex_time is True:
      execution_time = time.time() - start_time

      print(f"Execution time was {execution_time:.2f} seconds.")

    return W


## State Vector function

In [ ]:
import numpy as np

def update_state(F, conn, alpha, N, W, snpast, current_timestep,
                 input_vector=None, W_in=None, Laplacian=True ):
        # Initialize the new state vector
        sn = np.zeros_like(snpast)

        # Apply the chaotic map F element-wise to the previous state
        F_snpast = F(snpast)

        if conn == "Electrical":
          # Loop over each neuron to calculate its new state
          k_hat_vec = weighted_degrees_unsorted(W,N)
          # Replace zeros with inf to avoid division by zero
          k_hat_vec = np.where(k_hat_vec == 0, np.inf, k_hat_vec)

          if Laplacian == False:
            interaction_vec = None

            for i in range(N):
              interaction_sum =0
              for j in range(N):

                  interaction = W[i, j] * (snpast[j] - snpast[i])
                  interaction_sum += (interaction)

              if interaction_vec is None:
                interaction_vec = interaction_sum
              else:
                interaction_vec = np.row_stack((interaction_vec,interaction_sum))

            if input_vector is not None and W_in is not None:
              input = np.dot(W_in,input_vector)
              interaction_vec += input

            sn = (1 - alpha) * F_snpast + alpha * interaction_vec

          else:
            L = calculate_laplacian(W)
            if input_vector is not None and W_in is not None:
              input = np.dot(W_in,input_vector)
              sn = (1 - alpha) * F_snpast + alpha * np.dot(1/(2*k_hat_vec), (np.dot(L,snpast) + input))
            else:
              sn = (1 - alpha) * F_snpast + alpha * np.dot(1/(2*k_hat_vec), (np.dot(L,snpast)))


        elif conn == "Chemical":
          if input_vector is None:
            sn = (1-alpha)*F_snpast + alpha * np.tanh(np.dot(W, snpast))
          else:
            sn = (1-alpha)*F_snpast + alpha * np.tanh(np.dot(W, snpast) + np.dot(W_in,input_vector))

        return sn




## Network with only state matrix

In [ ]:
import numpy as np
import scipy.sparse.linalg as sp
import time

def compute_state_matrix(F, connectivity, N, epsilon, rho, alpha, T,
                         Lapl=True, ex_time=False):


    # Set the seed for reproducability
    np.random.seed(157)

    # Start the timer
    start_time = time.time()

    # Initialize W using the initialize_reservoir function
    W = initialize_reservoir(N, epsilon, rho)

    count = 10  # variable to keep track of progress in increments of 5 percent
    snpast = -1 + 2 * np.random.rand(N, 1)  # state vecto
    X = snpast  # state matrix initialized as None


    # Doing the calculations
    for t in range(T-1):


        # Update equation
        # update_state(F, conn, alpha, N, W, snpast, current_timestep,
                    #  input_vector=None, W_in=None, Laplacian=True )
        sn = update_state(F,connectivity, alpha, N, W, snpast, t,
                          input_vector=None, W_in=None, Laplacian=Lapl)

        X = np.column_stack((X, sn))

        snpast = sn  # Changing sn to snpast

        if ex_time is True:
          percent = round(t / T * 100, 1)  # Show progress every five percent
          if percent == count:
              print(f"Progress: {percent:.1f} Percent.")
              count += 5

    # Calculate execution time
    if ex_time is True:
      execution_time = time.time() - start_time
      print(f"Execution time was {execution_time:.2f} seconds.")

    return X, N, epsilon, rho, T, alpha, W

## Input Layer

In [ ]:
def create_w_in(N_input, N_u=1):
    import numpy as np
    # We set seed so results can be reproducable
    np.random.seed(157)

    # Create a (NxN_u+1) matrix with values in [-0.5, 0.5]
    W_in = (-1+ (2*1) * np.random.rand(N_input, N_u+1))*0.5
    return W_in

In [ ]:
import numpy as np
import scipy.sparse.linalg as sp
import time

def compute_state_matrix_w_input(N, epsilon, rho, T, alpha, input_data, bias, F=F_identity, N_input=10, N_u=1, ex_time=False):

    # Set the seed for reproducability
    np.random.seed(157)

    # Start the timer
    start_time = time.time()
    count = 10  # variable to keep track of progress in increments of 5 percent

    # Experimentation with number of neurons connected with input layer
    N_input=N
    # Initialize W using the initialize_reservoir function
    W = initialize_reservoir(N, epsilon, rho)
    # Create input matrix
    W_in = create_w_in(N_input, N_u)
    # Make bias into numpy array
    b = np.array(bias)

    # state vector
    snpast = -1 + 2 * np.random.rand(N, 1)
    # state matrix initialized as None
    X = None

    # Doing the calculations
    for t in range(T):

        # Get the Lorenz system input for the current time step
        u = np.array(input_data[t])
        # Stack b on top of u to create a column vector
        input_vector = np.vstack((b, u))



        sn = update_state(N, alpha, W, snpast, F, input_vector, W_in)

        # Store the state in the matrix X
        if X is None:  # if X is "zero" (or None in this context)
            X = sn
        else:
            X = np.column_stack((X, sn))

        snpast = sn  # Changing sn to snpast

        if ex_time is True:
          percent = round(t / T * 100, 1)  # Show progress every five percent
          if percent == count:
              print(f"Progress: {percent:.1f} Percent.")
              count += 5

    # Calculate execution time
    if ex_time is True:
      execution_time = time.time() - start_time
      print(f"Execution time was {execution_time:.2f} seconds.")

    return X, N, epsilon, rho, T, alpha

## Output Layer

In [ ]:
def calculate_W_out(X,Y,regulization):
  # Calculate X transpose
  X_T = X.T

  # Calculate term inside parenthesis
  term = X @ X_T + regulization**2 * np.eye(X.shape[0])

  # Calculate the pseudo-inverse of the term
  term_pseudo_inv = np.linalg.pinv(term)

  # Finally, calculate W_out
  W_out = Y @ X_T @ term_pseudo_inv

  return W_out

In [ ]:
def state_matrix(n,alpha, b_in, b_out, W_in, W, X, snpast, input_data, F=F_identity):
            # Get the Lorenz system input for the current time step
          if np.array(input_data.shape[1]) != 1:
            un = np.array(input_data[:,n]).reshape(-1,1)
          else:
            un = np.array(input_data).reshape(-1,1)

          # Stack b on top of u to create a column vector
          input_vector = np.vstack((b_in, un))


          sn = update_state(N, alpha, W, snpast, F, input_vector, W_in)

          xn = np.row_stack((b_out, sn, un))

          if X is None:  # if X is "zero" (or None in this context)
            X = xn
          else:
            X = np.column_stack((X, xn))

          snpast = sn  # Changing sn to snpast

          return X, snpast, xn

## Looped Network

In [ ]:
import numpy as np
import scipy.sparse.linalg as sp
import time

def looped_network(F, connectivity, input_data, output_data,
            N, epsilon, rho, alpha,
            T_transient, T_training, T_prediction, tau,
            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

    # Set the seed for reproducability
    np.random.seed(157)

    # Start the timer
    start_time = time.time()
    count = 10  # variable to keep track of progress in increments of 5 percent

    # Increments Reservoir will be allowed to run
    T = T_transient+T_training+T_prediction
    # Initialize W using the initialize_reservoir function
    W = initialize_reservoir(N, epsilon, rho)
    # Calculate number of input neurons
    N_u = input_data.shape[0]
    # Create input matrix
    W_in = create_w_in(N, N_u)
    # Make bias into numpy array
    b_in = np.array(b_in)
    b_out = np.array(b_out)

    # state vector
    snpast = -1 + 2 * np.random.rand(N, 1)
    # state matrix initialized as None
    X = None
    # Prediction matrix
    Y = None


    # Doing the calculations
    for n in range(T):
        # Get the Lorenz system input for the current time step
        un = np.array(input_data[:,n]).reshape(-1,1)

        # Stack b on top of u to create a column vector
        input_vector = np.vstack((b_in, un))

        # For training and transient input the RK4 integration
        if n < (T_transient):
          sn = update_state(F, connectivity, alpha, N, W, snpast, n,
                            input_vector=input_vector, W_in=W_in, Laplacian=Lapl)
          xn = np.row_stack((b_out, sn, un))
          if X is None:  # if X is "zero" (or None in this context)
            X = xn
          else:
            X = np.column_stack((X, xn))
          snpast = sn
        # Training of the Network
        elif n <(T_transient+T_training):
          sn = update_state(F, connectivity, alpha, N, W, snpast, n,
                          input_vector=input_vector, W_in=W_in, Laplacian=Lapl)
          xn = np.row_stack((b_out, sn, un))
          X = np.column_stack((X, xn))
          snpast = sn

        elif n == (T_training + T_transient):
                                        # Get the Lorenz system input for the current time step
          sn = update_state(F, connectivity, alpha, N, W, snpast, n,
                          input_vector=input_vector, W_in=W_in, Laplacian=Lapl)
          xn = np.row_stack((b_out, sn, un))
          X = np.column_stack((X, xn))
          # Calculate W_out
          if output_data.shape[0] == 1:
                W_out = calculate_W_out(X[:,T_transient:(T_transient+T_training)],
                                output_data[T_transient+tau:(T_transient+T_training+tau)],regulization)
          else:
                W_out = calculate_W_out(X[:,T_transient:(T_transient+T_training)],
                                output_data[:,T_transient+tau:(T_transient+T_training+tau)],regulization)

          # prediction vector
          yn = np.dot(W_out,xn)
          Y = yn
          ynpast = yn
          snpast = sn


        #n <= (T)

        else:
          if N_u == ynpast.shape[0]:
            input_vector = np.vstack((b_in, ynpast))
            sn = update_state(F, connectivity, alpha, N, W, snpast, n,
                          input_vector=input_vector, W_in=W_in, Laplacian=Lapl)
            xn = np.row_stack((b_out, sn, ynpast))
            X = np.column_stack((X, xn))


            # prediction vector
            yn = np.dot(W_out,xn)
            Y = np.column_stack((Y, yn))
            ynpast = yn
            snpast = sn


          else:
            raise Exception("Number of input neurons does not meet the looped prediction neurons")


        if ex_time is True:
          percent = round(n / T * 100, 1)  # Show progress every five percent
          if percent == count:
              print(f"Progress: {percent:.1f} Percent.")
              count += 5

    # Calculate execution time
    if ex_time is True:
      execution_time = time.time() - start_time
      print(f"Execution time was {execution_time:.2f} seconds.")

    return X, N, epsilon, rho, T_transient, T_training, T_prediction, alpha, Y, tau, W


## Final Network

In [ ]:
import numpy as np
import scipy.sparse.linalg as sp
import time

def network(F, connectivity, input_data, output_data,
            N, epsilon, rho, alpha,
            T_transient, T_training, T_prediction, tau,
            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

    # Set the seed for reproducability
    np.random.seed(157)

    # Start the timer
    start_time = time.time()
    count = 10  # variable to keep track of progress in increments of 5 percent

    # Increments Reservoir will be allowed to run
    T = T_transient+T_training+T_prediction
    # Initialize W using the initialize_reservoir function
    W = initialize_reservoir(N, epsilon, rho)
    # Calculate number of input neurons
    N_u = input_data.shape[0]
    # Create input matrix
    W_in = create_w_in(N, N_u)
    # Make bias into numpy array
    b_in = np.array(b_in)
    b_out = np.array(b_out)

    # state vector
    snpast = -1 + 2 * np.random.rand(N, 1)
    # state matrix initialized as None
    X = None
    # Prediction matrix
    Y = None

    # Doing the calculations
    for n in range(T):

        # Get the Lorenz system input for the current time step
        un = np.array(input_data[:,n]).reshape(-1,1)

        # Stack b on top of u to create a column vector
        input_vector = np.vstack((b_in, un))


        # state vector equation
        # update_state(F, conn, alpha, N, W, snpast, current_timestep,
                    #  input_vector=None, W_in=None, Laplacian=True )
        sn = update_state(F, connectivity, alpha, N, W, snpast, n,
                          input_vector=input_vector, W_in=W_in, Laplacian=Lapl)

        xn = np.row_stack((b_out, sn, un))
        if X is None:  # if X is "zero" (or None in this context)
          X = xn
        else:
          X = np.column_stack((X, xn))

        # Training of the Network
        if n == (T_training + T_transient):
            if output_data.shape[0] == 1:
              W_out = calculate_W_out(X[:,T_transient:(T_transient+T_training)],
                              output_data[(T_transient+tau):(T_transient+T_training+tau)],regulization)
            else:
              W_out = calculate_W_out(X[:,(T_transient):(T_transient+T_training)],
                              output_data[:,(T_transient+tau):(T_transient+T_training+tau)],regulization)


        if n >=(T_training+T_transient):

          # prediction vector
          yn = np.dot(W_out,xn)

          if Y is None:
            Y = yn
          else:
            Y = np.column_stack((Y, yn))

        snpast = sn  # Changing sn to snpast

        if ex_time is True:
          percent = round(n / T * 100, 1)  # Show progress every five percent
          if percent == count:
              print(f"Progress: {percent:.1f} Percent.")
              count += 5

    # Calculate execution time
    if ex_time is True:
      execution_time = time.time() - start_time
      print(f"Execution time was {execution_time:.2f} seconds.")

    return X, N, epsilon, rho, T_transient, T_training, T_prediction, alpha, Y, tau, W

# Analysis Functions




### NN Convergence


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def computational_units_state(X, legend=False, y_lim=[-1,1], tick_labelsize=18, axis_labelsize=18):
    # Create a vector for the x-axis representing T increments
    T = X.shape[1]
    N = X.shape[0]
    t = np.arange(0, T)

    # Plot each row of X as a separate line
    plt.figure(figsize=(12, 6))
    for i in range(N):
      if legend:
          plt.plot(t, X[i, :], label=f'Neuron {i+1}')  # Adding a label for each neuron
      else:
          plt.plot(t, X[i, :])  # Plot without a label if legend is False
    # Set y-axis limits
    plt.ylim(y_lim)

    # Set the tick parameters for both axes to increase tick label size
    plt.tick_params(axis='both', labelsize=tick_labelsize)

        # Label the axes and set their font size
    plt.xlabel('Number of timesteps', fontsize=axis_labelsize)
    plt.ylabel('State of Neurons', fontsize=axis_labelsize)


    # Show the plot
    plt.show()

# Example usage (assuming X is your state matrix, N is the number of neurons, and T is the number of timesteps)
# computational_units_state(X, N, T)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def computational_units_state_2(X, nn, t_start, t_end, y_lim=None, zoom_in_start=None, zoom_in_end=None, legend=False,tick_labelsize=18, axis_labelsize=18):
    # Adjust the range of neurons and timesteps based on provided arguments
    X_zoomed = X[0:nn, t_start:t_end]
    N = X_zoomed.shape[0]
    T = X_zoomed.shape[1]
    t = np.arange(t_start, t_end)

    # Determine y-axis limits based on the first neuron's data range
    if y_lim is None:
      y_lim = [min(X[0,:]), -min(X[0,:])]

    # Plot each row of X_zoomed as a separate line
    plt.figure(figsize=(12, 6))
    for i in range(N):
        if legend:
            plt.plot(t, X_zoomed[i, :], label=f'Neuron {i+1}')  # Adding a label for each neuron
        else:
            plt.plot(t, X_zoomed[i, :])  # Plot without a label if legend is False

    # Set y-axis limits
    plt.ylim(y_lim)

    # Set x-axis limits for zoomed-in view if specified
    if zoom_in_start is not None and zoom_in_end is not None:
        plt.xlim([zoom_in_start, zoom_in_end])

    # Set the tick parameters for both axes to increase tick label size
    plt.tick_params(axis='both', labelsize=tick_labelsize)

        # Label the axes and set their font size
    plt.xlabel('Number of timesteps', fontsize=axis_labelsize)
    plt.ylabel('State of Neurons', fontsize=axis_labelsize)



    # Show the plot
    plt.show()



### Plot of State Average

In [ ]:
def states_average_plot_beta(X, t_transient=0, t_end=None):
    import numpy as np
    import matplotlib.pyplot as plt
    T = X.shape[1]
    N = X.shape[0]
    # If t_end is not provided, use the last timestep
    if t_end is None:
        t_end = X.shape[1]

    # Average of X
    Xnotransient = X[:, t_transient:t_end]  # Python uses 0-based indexing
    k = np.mean(Xnotransient, axis=0)

    # Create a vector for the x-axis representing T increments
    t = np.arange(t_transient, t_end)

    # Plot average of states
    plt.figure(figsize=(12,6))
    plt.plot(t, k)
    plt.xlabel('Timestep')
    plt.ylabel('Average of states')
    plt.title('Average of states')
    plt.show()





In [ ]:
def calculate_running_average(data, window_size):
    running_avg = []
    for i in range(len(data)):
        if i < window_size - 1:
            # Not enough data points for the window size yet
            average = sum(data[:i+1]) / (i+1)
        else:
            # Calculate the sum of the current window
            window_sum = sum(data[i-window_size+1:i+1])
            average = window_sum / window_size
        running_avg.append(average)
    return running_avg

In [ ]:
def states_average_plot(X, t_transient=0, t_end=None, window_size=10, running_avg=False,tick_labelsize=18, axis_labelsize=18):
    # This is updated version with running average
    import numpy as np
    import matplotlib.pyplot as plt

    T = X.shape[1]
    N = X.shape[0]
    # If t_end is not provided, use the last timestep
    if t_end is None:
        t_end = X.shape[1]

    # Average of X
    Xnotransient = X[:, t_transient:t_end]  # Python uses 0-based indexing
    k = np.mean(Xnotransient, axis=0)

    # Create a vector for the x-axis representing T increments
    t = np.arange(t_transient, t_end)

    if running_avg:
      # Compute the running average
      run_avg = calculate_running_average(k, window_size)
      t_run_avg = t[:len(run_avg)]  # This slices the t array to match the length of run_avg


    # Plot average of states
    plt.figure(figsize=(12,6))
    plt.plot(t, k, label='Average')
    if running_avg:
      plt.plot(t_run_avg, run_avg, label='Running Average', color='red')
        # Set the tick parameters for both axes to increase tick label size
    plt.tick_params(axis='both', labelsize=tick_labelsize)

            # Label the axes and set their font size
    plt.xlabel('Number of timesteps', fontsize=axis_labelsize)
    plt.ylabel('Average of states', fontsize=axis_labelsize)

    plt.legend()
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def states_average_plot_zoom_in(X, t_start, t_zoom_end, t_transient=0, t_end=None, window_size=10, running_avg=False,tick_labelsize=18, axis_labelsize=18):
    # This is the updated version with running average and zoom-in feature
    # If t_end is not provided, use the last timestep

    T = X.shape[1]
    N = X.shape[0]
    if t_end is None:
        t_end = X.shape[1]

    # Average of X
    Xnotransient = X[:, t_transient:t_end]  # Python uses 0-based indexing
    k = np.mean(Xnotransient, axis=0)

    # Create a vector for the x-axis representing T increments
    t = np.arange(t_transient, t_end)

    if running_avg:
        # Compute the running average
        run_avg = calculate_running_average(k, window_size)
        t_run_avg = t[:len(run_avg)]  # This slices the t array to match the length of run_avg

    # Plot average of states
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(t, k, label='Average')
    if running_avg:
        ax.plot(t_run_avg, run_avg, label='Running Average', color='red')

    plt.tick_params(axis='both', labelsize=tick_labelsize)
    plt.xlabel('Number of timesteps', fontsize=axis_labelsize)
    plt.ylabel('Average of states', fontsize=axis_labelsize)

    # ax.set_title('Average of states with Running Average')
    ax.legend()

    # Create zoomed-in inset
    axins = inset_axes(ax, width="30%", height="30%", loc='upper right', borderpad=1)
    axins.plot(t, k, label='Average')
    if running_avg:
        axins.plot(t_run_avg, run_avg, label='Running Average', color='red')


    # Specify the limits for the zoomed-in region here (example: from timestep 20 to 40)
    axins.set_xlim(t_start, t_zoom_end)  # Adjust these values based on your data
    axins.set_ylim(min(k[t_start:t_zoom_end]), max(k[t_start:t_zoom_end]))  # Set the y-limits to match the zoom

    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def states_average_plot_w_input_data(X, N, T,input_data, input_name, output_name="Average States", title=None, t_transient=0, t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200)):
    # If t_end is not provided, use the last timestep

    T = X.shape[1]
    N = X.shape[0]
    if t_end is None:
      if X.ndim == 1:
        t_end = X.shape[0]
      else:
        t_end = X.shape[1]

    # Average of X
    Xnotransient = X[:, t_transient:t_end]  # Python uses 0-based indexing
    k = np.mean(Xnotransient, axis=0)


    # Normalize Lorenz data
    input_mean = np.mean(input_data)
    input_std = np.std(input_data)
    k_std = np.std(k)
    input_norm = (input_data - input_mean) / input_std * k_std
    input_norm_not_transient = input_norm[t_transient:t_end]

    # Create a vector for the x-axis representing T increments
    t = np.arange(t_transient, t_end)
    t_input = np.arange(t_transient, t_end)  # Assuming Lorenz data has the same timestep as X

    if running_avg:
        # Compute the running average
        run_avg = calculate_running_average(k, window_size)
        t_run_avg = t[:len(run_avg)]  # This slices the t array to match the length of run_avg

    # Main plot
    fig, ax = plt.subplots(figsize=(12,6))
    ax.plot(t, k, label=f'{output_name}')
    if running_avg:
        ax.plot(t_run_avg, run_avg, label='Running Average', color='red')
    ax.plot(t_input, input_norm_not_transient, label=f'Normalized {input_name}', color='green', alpha=0.5)

    if title is None:
      ax.set_title(f'{output_name} with Normalized {input_name}')
    else:
      ax.set_title(f'{title}')
    ax.set_xlabel('Timestep')
    ax.set_ylabel('Normalized Values')
    ax.legend(loc=2)


    if zoom_in:
      # Inset plot for zoom-in
      ax_inset = inset_axes(ax, width="30%", height="30%", loc=1)  # loc=1 is upper right
      ax_inset.plot(t[zoom_range[0]:zoom_range[1]], k[zoom_range[0]:zoom_range[1]], color='blue')
      ax_inset.plot(t_input[zoom_range[0]:zoom_range[1]], input_norm_not_transient[zoom_range[0]:zoom_range[1]], color='green', alpha=0.5)
      ax_inset.set_xlim(zoom_range[0], zoom_range[1])
      ax_inset.set_title("Zoomed In")

    plt.show()


In [ ]:
# No normalization

def states_average_plot_w_input_data_2(X, N, T,input_data, input_name, output_name="Average States", title=None, t_transient=0, t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200)):
    # If t_end is not provided, use the last timestep

    T = X.shape[1]
    N = X.shape[0]
    if t_end is None:
      if X.ndim == 1:
        t_end = X.shape[0]
      else:
        t_end = X.shape[1]

    # Average of X
    Xnotransient = X[:, t_transient:t_end]  # Python uses 0-based indexing
    k = np.mean(Xnotransient, axis=0)

    input_norm_not_transient = input_data[t_transient:t_end]

    # Create a vector for the x-axis representing T increments
    t = np.arange(t_transient, t_end)
    t_input = np.arange(t_transient, t_end)  # Assuming Lorenz data has the same timestep as X

    if running_avg:
        # Compute the running average
        run_avg = calculate_running_average(k, window_size)
        t_run_avg = t[:len(run_avg)]  # This slices the t array to match the length of run_avg

    # Main plot
    fig, ax = plt.subplots(figsize=(10,6))
    ax.plot(t, k, label=f'{output_name}')
    if running_avg:
        ax.plot(t_run_avg, run_avg, label='Running Average', color='red')
    ax.plot(t_input, input_norm_not_transient, label=f'{input_name}', color='green', alpha=0.5)

    #if title is None:
    #  ax.set_title(f'{output_name} with {input_name}')
    #else:
   #   ax.set_title(f'{title}')
    ax.set_xlabel('Timestep', fontsize=16)
    ax.legend(loc=2, fontsize=16)
    plt.tick_params(axis='both', labelsize=16)

    if zoom_in:
      # Inset plot for zoom-in
      ax_inset = inset_axes(ax, width="30%", height="30%", loc=1)  # loc=1 is upper right
      ax_inset.plot(t[zoom_range[0]:zoom_range[1]], k[zoom_range[0]:zoom_range[1]], color='blue')
      ax_inset.plot(t_input[zoom_range[0]:zoom_range[1]], input_norm_not_transient[zoom_range[0]:zoom_range[1]], color='green', alpha=0.5)
      ax_inset.set_xlim(zoom_range[0], zoom_range[1])
      ax_inset.set_title("Zoomed In")

    plt.show()

In [ ]:
# From Reservoir 2.8
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def states_average_plot_w_input_data_3(X, N, T,input_data, input_name, output_name="Average States", title=None, t_transient=0, t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200)):
    # If t_end is not provided, use the last timestep
    if t_end is None:
      if X.ndim == 1:
        t_end = X.shape[0]
      else:
        t_end = X.shape[1]

    # Average of X
    Xnotransient = X[:, t_transient:t_end]  # Python uses 0-based indexing
    k = np.mean(Xnotransient, axis=0)


    # Normalize Lorenz data
    input_mean = np.mean(input_data)
    input_std = np.std(input_data)
    k_std = np.std(k)
    input_norm = (input_data - input_mean) / input_std * k_std
    input_norm_not_transient = input_norm[t_transient:t_end]

    # Create a vector for the x-axis representing T increments
    t = np.arange(t_transient, t_end)
    t_input = np.arange(t_transient, t_end)  # Assuming Lorenz data has the same timestep as X

    if running_avg:
        # Compute the running average
        run_avg = calculate_running_average(k, window_size)
        t_run_avg = t[:len(run_avg)]  # This slices the t array to match the length of run_avg

    # Main plot
    fig, ax = plt.subplots(figsize=(12,6))
    ax.plot(t, k, label=f'{output_name}')
    if running_avg:
        ax.plot(t_run_avg, run_avg, label='Running Average', color='red')
    ax.plot(t_input, input_norm_not_transient, label=f'Normalized {input_name}', color='green', alpha=0.5)

    if title is None:
      ax.set_title(f'{output_name} with Normalized {input_name}')
    else:
      ax.set_title(f'{title}')
    ax.set_xlabel('Timestep')
    ax.set_ylabel('Normalized Values')
    ax.legend(loc=2)
    ax.grid(True)

    if zoom_in:
      # Inset plot for zoom-in
      ax_inset = inset_axes(ax, width="30%", height="30%", loc=1)  # loc=1 is upper right
      ax_inset.plot(t[zoom_range[0]:zoom_range[1]], k[zoom_range[0]:zoom_range[1]], color='blue')
      ax_inset.plot(t_input[zoom_range[0]:zoom_range[1]], input_norm_not_transient[zoom_range[0]:zoom_range[1]], color='green', alpha=0.5)
      ax_inset.set_xlim(zoom_range[0], zoom_range[1])
      ax_inset.set_title("Zoomed In")

    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def states_average_prediction(X, input_data, input_name, output_name="Average States", title=None, t_transient=0, window_size=10, zoom_in=False, zoom_range=(100, 200),tick_labelsize=18, axis_labelsize=18):

    T = X.shape[1]
    N = X.shape[0]
    k = np.mean(X, axis=0)
    t_end = X.shape[0]
    # Create a vector for the x-axis representing T increments
    t = np.arange(0, t_end)


    # Main plot
    fig, ax = plt.subplots(figsize=(12,6))
    ax.plot(t, k, label=f'{output_name}')
    ax.plot(t, input_data, label=f' {input_name}', color='green', alpha=0.5)

    if title is None:
      ax.set_title(f'{output_name} with {input_name}')
    else:
      ax.set_title(f'{title}')

    plt.tick_params(axis='both', labelsize=tick_labelsize)
    plt.xlabel('Number of timesteps', fontsize=axis_labelsize)
    plt.ylabel('Average of states', fontsize=axis_labelsize)
    ax.legend(loc=2)

    if zoom_in:
      # Inset plot for zoom-in
      ax_inset = inset_axes(ax, width="30%", height="30%", loc=1)  # loc=1 is upper right
      ax_inset.plot(t[zoom_range[0]:zoom_range[1]], k[zoom_range[0]:zoom_range[1]], color='blue')
      ax_inset.plot(t[zoom_range[0]:zoom_range[1]], input_data[zoom_range[0]:zoom_range[1]], color='green', alpha=0.5)
      ax_inset.set_xlim(zoom_range[0], zoom_range[1])
      ax_inset.set_title("Zoomed In")

    plt.show()


### Plot of Variance

In [ ]:
# Variance plot
def states_variance_plot(X, t_transient=0, t_end=None, tick_labelsize=18, axis_labelsize=18):
    import numpy as np
    import matplotlib.pyplot as plt

    T = X.shape[1]
    N = X.shape[0]
    # If t_end is not provided, use the last timestep
    if t_end is None:
        t_end = X.shape[1]

    # Extract the non-transient part of X
    Xnotransient = X[:, t_transient:t_end]  # Python uses 0-based indexing
    v = np.var(Xnotransient, axis=0)

    # Adjust the range of t to match the length of v
    t = np.arange(t_transient, t_end)

    plt.figure(figsize=(12,6))
    plt.plot(t, v, linewidth=0.5)
    plt.tick_params(axis='both', labelsize=tick_labelsize)
    plt.xlabel('Number of timesteps', fontsize=axis_labelsize)
    plt.ylabel('Variance of states', fontsize=axis_labelsize)
    plt.show()



### Integrals

In [ ]:
def integral_of_spikes_old(X, treshold=10, window_size=10, running_avg= False, integral_analysis=False, f_min=None, f_max=None):

    T = X.shape[1]

    # Average of X
    Xnotransient = X  # Assuming you want the entire matrix
    k = np.mean(Xnotransient, axis=0)

    # Compute the running average using the new function
    if running_avg:
        k = calculate_running_average(k, window_size)

    # Shifted FFT
    B_shift = np.fft.fftshift(np.fft.fft(k))
    B_mag_shift = np.abs(B_shift)

    # Create a frequency vector
    n = len(k)  # Length of the signal
    Fs = n / T  # Sampling frequency based on the provided T
    dF = Fs / n  # Frequency resolution: 0.0001 if T=10000
    f_values = np.arange(-Fs/2, Fs/2, dF)  # Frequency vector centered around zero

    # Determine the indices corresponding to f_min and f_max
    if f_min is not None:
        idx_min = np.where(f_values >= f_min)[0][0]
    else:
        idx_min = 0

    if f_max is not None:
        idx_max = np.where(f_values <= f_max)[0][-1]
    else:
        idx_max = len(f_values) - 1

    spike_count = 0
    integrals = []
    area_sum = 0

    for i in range(idx_min, idx_max + 1):
        amplitude = 0
        if B_mag_shift[i] > treshold:
            spike_count += 1
            amplitude = B_mag_shift[i] - treshold
            area_sum += amplitude*dF

        integrals.append(amplitude * dF)


    if integral_analysis:
      return area_sum, integrals, spike_count
    else:
      return area_sum


In [ ]:
def integral_of_spikes(X, threshold=10, window_size=10, running_avg= False, integral_analysis=False, f_min=None, f_max=None):

    T = X.shape[1]

    # Average of X
    Xnotransient = X  # Assuming you want the entire matrix
    k = np.mean(Xnotransient, axis=0)

    # Compute the running average using the new function
    if running_avg:
        k = calculate_running_average(k, window_size)

    # Shifted FFT
    B_shift = np.fft.fftshift(np.fft.fft(k))
    B_mag_shift = np.abs(B_shift)

    # Create a frequency vector
    n = len(k)  # Length of the signal
    Fs = n / T  # Sampling frequency based on the provided T
    dF = Fs / n  # Frequency resolution: 0.0001 if T=10000
    f_values = np.arange(-Fs/2, Fs/2, dF)  # Frequency vector centered around zero

    # Determine the indices corresponding to f_min and f_max
    if f_min is not None:
        idx_min = np.where(f_values >= f_min)[0][0]
    else:
        idx_min = 0

    if f_max is not None:
        idx_max = np.where(f_values <= f_max)[0][-1]
    else:
        idx_max = len(f_values) - 1

    spike_count = 0
    integrals = []
    area_sum = 0

    for i in range(idx_min, idx_max + 1):
        amplitude = 0
        if B_mag_shift[i] > threshold:
            spike_count += 1
            amplitude = B_mag_shift[i] - threshold
            area_sum += amplitude*dF

        integrals.append(amplitude * dF)


    if integral_analysis:
      return area_sum, integrals, spike_count
    else:
      return area_sum

### Plot of FFT

In [ ]:
def fft_plot(X, threshold=10, t_transient=0, t_end=None, f_min=None, f_max=None, integrals=None, running_avg=False, window_size=10, tick_labelsize=18, axis_labelsize=18):
    # If t_end is not provided, use the last timestep
    if t_end is None:
        t_end = X.shape[1]
    T = X.shape[1]
    N = X.shape[0]
    # Average of X
    Xnotransient = X[:, t_transient:t_end]  # Python uses 0-based indexing
    k = np.mean(Xnotransient, axis=0)

    if running_avg:
        # Compute the running average using the new function
        k = calculate_running_average(k, window_size)

    # Shifted FFT
    B_shift = np.fft.fftshift(np.fft.fft(k))
    B_mag_shift = np.abs(B_shift)

    # Create a frequency vector
    n = len(k)  # Length of the signal
    Fs = n / T  # Sampling frequency based on the provided T
    dF = Fs / n  # Frequency resolution
    f = np.arange(-Fs/2, Fs/2, dF)  # Frequency vector centered around zero

    # Plot the magnitude of the shifted FFT
    plt.figure(figsize=(12,6))
    plt.plot(f, B_mag_shift, linewidth=0.5)

    # Add a horizontal line at the threshold value

    if integrals is not None:
        plt.axhline(y=threshold, color='r', linestyle='--', label=f'Threshold at {threshold}')
        for i in range(len(integrals)):
            if integrals[i] > 0:
                plt.fill_between(f[i:i+2], threshold, threshold + integrals[i]/dF, color='gray', alpha=0.5)

    if running_avg:
      method = 'Running Average FFT'
    else:
      method = 'FFT'
    plt.tick_params(axis='both', labelsize=tick_labelsize)
    plt.xlabel('Frequency (Hertz)', fontsize=axis_labelsize)
    plt.ylabel('Magnitude', fontsize=axis_labelsize)


    # If frequency limits are provided, adjust the x-axis limits
    if f_min is not None and f_max is not None:
        plt.xlim([f_min, f_max])

    # Add the legend
    plt.legend()

    plt.show()


### Plot of Power Spectrum

In [ ]:
# Power spectrum
def power_spectrum_plot(X, t_transient=0, t_end=None, f_min=None, f_max=None,tick_labelsize=18, axis_labelsize=18):
    import numpy as np
    import matplotlib.pyplot as plt

    T = X.shape[1]
    N = X.shape[0]
    # If t_end is not provided, use the last timestep
    if t_end is None:
        t_end = X.shape[1]

    # Average of X
    Xnotransient = X[:, t_transient:t_end]  # Python uses 0-based indexing
    k = np.mean(Xnotransient, axis=0)

    B_shift = np.fft.fftshift(np.fft.fft(k))

    # Create a frequency vector
    n = len(k)  # Length of the signal
    Fs = n / T  # Sampling frequency based on the provided T
    dF = Fs / n  # Frequency resolution
    f = np.arange(-Fs/2, Fs/2, dF)  # Frequency vector centered around zero

    # Power Spectral Density
    psdb = (1 / (Fs * n)) * np.abs(B_shift)**2

    # Power spectral density plotted logarithmically
    plt.figure(figsize=(12,6))
    plt.semilogy(f, psdb, linewidth=0.5)
    plt.tick_params(axis='both', labelsize=tick_labelsize)
    plt.xlabel('Frequency (Hertz)', fontsize=axis_labelsize)
    plt.ylabel('Log of Magnitude', fontsize=axis_labelsize)

    # If frequency limits are provided, adjust the x-axis limits
    if f_min is not None and f_max is not None:
        plt.xlim([f_min, f_max])

    plt.show()



### Pearson Correlation

In [ ]:
def pearson_cor(X, tau=10):

    # Average of X
    Xnotransient = X  # Assuming you want the entire matrix
    k = np.mean(Xnotransient, axis=0)
    k_2 = np.mean(k)

    sum = 0
    sum_2 = 0

    for i in range(len(k)-tau):
      averages_diff_now = k[i]-k_2 # difference between average of state vecors and average overall
      averages_diff_future = k[i+tau]-k_2 # difference between average of state vectors in tau increments and average overall
      sum += averages_diff_now * averages_diff_future

    # Variance^2
    for j in range(len(k)-tau):
      averages_diff_now_2 = k[j]-k_2
      sum_2 +=  averages_diff_now_2**2

    # Pearson Correlation
    p = abs(sum/sum_2)

    return p

In [ ]:
def pear_list(X,future_time=50):
  # Future time is what range of tau's we take. So in this case tau=1,2,3 ... 50
  tau_list = []
  pearson_list = []

  for tau in range(future_time):
    tau_list.append(tau)
    pearson_list.append(pearson_cor(X,tau))

  return pearson_list, tau_list

In [ ]:
def pear_avg(X, future_time=500, window_size=10):
    pearson_list, tau_list = pear_list(X,future_time)
    running_avg = calculate_running_average(pearson_list, window_size)

    # Adjust the tau_list for the running average
    adjusted_tau_list = tau_list[:len(running_avg)]
    return pearson_list, tau_list, running_avg, adjusted_tau_list

In [ ]:
def pear_graph(X, avg=False,integrals=None, treshold=0, future_time=500, window_size=10,tick_labelsize=18, axis_labelsize=18):
    import matplotlib.pyplot as plt

    pearson_list, tau_list, running_avg, adjusted_tau_list = pear_avg(X,future_time=future_time, window_size=10)
    dF=1/future_time
    f=np.arange(future_time)
    # Plot average of states
    plt.figure(figsize=(12,6))
    plt.plot(tau_list, pearson_list, label='Pearson Correlation')
    if avg:
      plt.plot(adjusted_tau_list, running_avg, label='Running Average', color='red')

    if integrals != None:
      for i in range(len(integrals)):
        if integrals[i] > 0:
            # Assuming tau_list contains the corresponding x-coordinates
            x_range = [tau_list[i], tau_list[i+1]] if i+1 < len(tau_list) else [tau_list[i], tau_list[i] + 1]
            plt.fill_between(x_range, [treshold, treshold], [treshold, treshold + integrals[i]], color='gray', alpha=0.75)

    plt.tick_params(axis='both', labelsize=tick_labelsize)
    plt.xlabel('Timesteps in future', fontsize=axis_labelsize)
    plt.ylabel('Pearson Correlation', fontsize=axis_labelsize)
    plt.legend()
    plt.show()

In [ ]:
def pear_in_ft(X, future_time=500, window_size=10):
   pearson_list, tau_list= pear_list(X,future_time)
   running_avg = calculate_running_average(pearson_list,window_size)
   return running_avg[-1]

In [ ]:
def pearson_integral(X, integral_analysis=True, treshold=0, future_time=500, window_size=10):
    pearson_list, tau_list, running_avg, adjusted_tau_list = pear_avg(X,future_time, window_size)
    dF = 1 / future_time  # Sampling frequency based on the provided T
    integrals = []
    area_sum = 0

    for i in range(future_time):
        amplitude = 0
        if running_avg[i] > treshold:
            amplitude = running_avg[i] - treshold
            area_sum += amplitude*1

        integrals.append(amplitude * 1)
    if integral_analysis:
      return area_sum, integrals
    else:
      return area_sum

### Grid Heatmap

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
def plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.2f'):
    # Calculate the median or mean value to set as the center of the colormap
    avg_val = np.average(data_matrix)

    if v_min is None and v_max is None:
      v_min = np.min(data_matrix)
      v_max = np.max(data_matrix)
    # Define a custom annotation function
    def fmt_func(x, pos):
        if x >= 1000:
            return f'{x/1000:.0f}k'
        elif x >= 10:
            return f'{x:.0f}'
        else:
            return f'{x:.1f}'
    # Center the colormap at the median value
    plt.figure(figsize=figsize)
    sns.heatmap(data_matrix, annot=True, cmap='viridis',
                xticklabels=x_labels, yticklabels=y_labels, fmt=fmt,
                vmin=v_min, vmax=v_max,
                cbar_kws={'format': FuncFormatter(fmt_func)})  # Adjust vmin and vmax if necessary

    plt.xlabel(f'{x_label_name} values')
    plt.ylabel(f'{y_label_name} values')


    plt.show()

### Grid function

In [ ]:
def grid(F, conn, T=1000, N=100, epsilon=0.2, rho=0.8, alpha=0.8, chaos_measure='fft integrals', treshold=10):
    # With this function you can grid search most chaotic hyperparameters
    S = []
    # The later parameter in list will be in rows.
    hyperparameters = {'N': N, 'epsilon': epsilon, 'rho': rho, 'alpha': alpha }
    par_to_grid = []

    for key, value in hyperparameters.items():
        if isinstance(value, list):
            par_to_grid.append((key, value))

    class CustomError(Exception):
        pass

    if len(par_to_grid) != 2:
        raise CustomError("This function accepts only 2 parameters as lists!")

    key1, var1 = par_to_grid[0]
    key2, var2 = par_to_grid[1]

    total_iterations = len(var1) * len(var2)
    current_iteration = 0

    for i in range(len(var1)):
        row = []
        for j in range(len(var2)):
            current_iteration += 1
            print(f"Processing {key1}={var1[i]}, {key2}={var2[j]} ... ({current_iteration}/{total_iterations})")

            # Update the hyperparameters dictionary
            hyperparameters[key1] = var1[i]
            hyperparameters[key2] = var2[j]

            # compute_state_matrix(F, connectivity, N, epsilon, rho, alpha, T,
                         #Lapl=True, ex_time=False):
            X, _, _, _, _, _, _ = compute_state_matrix(F, conn, **hyperparameters, T=T)
            if chaos_measure == 'fft integrals':
              row.append(integral_of_spikes(X, treshold))
            elif chaos_measure == 'spikes':
              # def integral_of_spikes(X, threshold=10, window_size=10, running_avg= False, integral_analysis=False, f_min=None, f_max=None)
              _,_,spikes = integral_of_spikes(X,5,10,False,True)
              row.append(spikes)
            elif chaos_measure == 'pearson':
              row.append(pear_in_ft(X, future_time=100))
            elif chaos_measure == 'pearson integrals':
              row.append(pearson_integral(X, integral_analysis=False))
            else:
              raise CustomError(f"{chaos_measure} is invalid: please specify -integrals- or -spikes- or -pearson-")
        S.append(row)

    return S


In [ ]:
def grid_prediction(input_data, train_data, T_transient=2000, T_training=5000, T_prediction=2000,
                    N=100, epsilon=0.2, rho=0.8, alpha=0.8, tau=1, b_in=1, b_out=1,
                    regulization=0, color='RMSE', F=F_identity):
    # With this function you can grid whatever you need for prediction
    # Color is written like that as it is color the heatmap will use
    # Here if grid is N and rho then
    # S rows will be wrt to rho
    # S columns will be wrt to N

    S = []
    hyperparameters = {'N': N, 'epsilon': epsilon, 'alpha': alpha, 'rho': rho }
    par_to_grid = []

    for key, value in hyperparameters.items():
        if isinstance(value, list):
            par_to_grid.append((key, value))

    class CustomError(Exception):
        pass

    if len(par_to_grid) != 2:
        raise CustomError("This function accepts only 2 parameters as lists!")

    key1, var1 = par_to_grid[0]
    key2, var2 = par_to_grid[1]

    total_iterations = len(var1) * len(var2)
    current_iteration = 0

    for i in range(len(var1)):
        row = []
        for j in range(len(var2)):
            current_iteration += 1
            print(f"Processing {key1}={var1[i]}, {key2}={var2[j]} ... ({current_iteration}/{total_iterations})")

            # Update the hyperparameters dictionary
            hyperparameters[key1] = var1[i]
            hyperparameters[key2] = var2[j]

            # Unpack the dictionary when calling the function
            X,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y, tau, W = network(
                **hyperparameters, T_transient=T_transient, T_training=T_training, T_prediction=T_prediction,
                tau=tau,
                input_data=input_data, output_data=train_data, b_in=b_in, b_out=b_out, regulization=regulization, F=F)
            """
            if color == 'integrals':
              row.append(integral_of_spikes(X, T, treshold))
            elif color == 'spikes':
              row.append(spike_count(X,T,treshold))
            elif color == 'pearson':
              row.append(pear_in_ft(X, future_time=100))
            elif color == 'pearson integrals':
              row.append(pearson_integral(X, integral_analysis=False))
            """
            if color == 'RMSE':
              overall_rmse, _ = rmse(input_data[0,(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)], Y[0,:])
              row.append(overall_rmse)
            else:
              raise CustomError(f"{color} is invalid: please specify -integrals- or -spikes- or -pearson- or -RMSE-")
        S.append(row)

    return S


### Simulation of dynamical systems

In [ ]:
def lorenz_system(t, x, y ,z, a, b, c):
    import numpy as np
    dxdt = -a * x + a * y
    dydt = c * x - y - x * z
    dzdt = -b * z + x * y
    return np.array([dxdt, dydt, dzdt])

In [ ]:
def rossler_system(t, x, y, z, a, b, c):
    dx = -y - z
    dy = x + a*y
    dz = b + z*(x-c)
    return np.array([dx, dy, dz])

def euler_method(f, t0, x0, y0, z0, a, b, c, h, steps):
    t = t0
    x, y, z = x0, y0, z0
    trajectory = [(x, y, z)]

    for _ in range(steps):
        dx, dy, dz = f(t, x, y, z, a, b, c)
        x += h * dx
        y += h * dy
        z += h * dz
        trajectory.append((x, y, z))
        t += h  # Increment the time

    return trajectory

In [ ]:
def rk4(f, t0, x0, y0, z0, a, b, c, h, steps, sampling_rate=1):
    t = t0
    w = np.array([x0, y0, z0])
    trajectory = [w]

    for _ in range(0,steps-1):
        k1 = h * f(t, *w, a, b, c)
        k2 = h * f(t + 0.5*h, *(w + 0.5*k1), a, b, c)
        k3 = h * f(t + 0.5*h, *(w + 0.5*k2), a, b, c)
        k4 = h * f(t + h, *(w + k3), a, b, c)

        w = w + (k1 + 2*k2 + 2*k3 + k4) / 6.0
        trajectory.append(w)
        t += h
    if sampling_rate!=1:
      sampling_trajectory=[]
      for i in range(0,steps-1,sampling_rate):
        sampling_trajectory.append(trajectory[i])
      return sampling_trajectory
    else:
      return trajectory

In [ ]:
def dynamical_syst_graph(*args):
    import matplotlib.pyplot as plt
    num_dimensions = len(args)

    # Create a figure
    fig = plt.figure(figsize=(10, 8))

    if num_dimensions == 1:
        # 1D plot
        plt.plot(args[0], lw=0.5, color='blue')
        plt.xlabel("Time Steps", color='black')
        plt.ylabel("X", color='black')
        plt.title("1D Dynamical System", color='black')

    elif num_dimensions == 2:
        # 2D plot
        plt.plot(args[0], args[1], lw=0.5, color='black')
        plt.xlabel("X", color='black', fontsize=16)
        plt.ylabel("Y", color='black', fontsize=16)
        #plt.title("", color='black')
        plt.tick_params(axis='both', labelsize=16)

    elif num_dimensions == 3:
        # 3D plot
        ax = fig.add_subplot(111, projection='3d', frame_on=False)
        ax.plot(args[0], args[1], args[2], lw=0.5, color='black')

        # Add a dot for the initial conditions
        ax.scatter([args[0][0]], [args[1][0]], [args[2][0]], color='red', s=25, label='Initial Condition')
        ax.legend()

        plt.tick_params(labelsize=12)
        ax.set_xlabel("X", color='black', fontsize=16)
        ax.set_ylabel("Y", color='black', fontsize=16)
        ax.set_zlabel("Z", color='black', fontsize=12)
        #ax.set_title("3D Dynamical System", color='black')

    else:
        raise ValueError("Unsupported number of dimensions. Please provide 1, 2, or 3 dimensions.")

    plt.show()

In [ ]:
def dynamical_syst_interactive_graph(xs,ys,zs, method, name):
  import plotly.graph_objects as go


  # Create the 3D plot
  fig = go.Figure(data=[go.Scatter3d(x=xs, y=ys, z=zs, mode='lines')])

  # Set the title and axis labels
  fig.update_layout(scene=dict(
                      xaxis_title='X Axis',
                      yaxis_title='Y Axis',
                      zaxis_title='Z Axis'),
                    title=f"{name} using {method} Method")

  # Show the plot
  fig.show()

### Root mean squared error function

In [ ]:
import math

def rmse(real, predicted):
    # Ensure real and predicted have the same length
    if len(real) != len(predicted):
        raise ValueError(f"The length of real{len(real)} and predicted {len(predicted)} must be the same")

    # Calculate the RMSE for each prediction and overall RMSE
    n = len(real)
    individual_errors = []
    error_sum = 0

    for r, p in zip(real, predicted):
        error = (r - p) ** 2
        individual_errors.append(math.sqrt(error))
        error_sum += error

    overall_rmse = math.sqrt(error_sum / n)

    return overall_rmse, individual_errors

### Degree investigation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def degrees_distribution(W):
  # Count nonzero entries in each column
  nonzero_counts = np.count_nonzero(W, axis=1)

  # Convert numpy array to a list and save it into 'degrees'
  degrees = nonzero_counts.tolist()

  # Draw the distribution plot
  sns.histplot(degrees, kde=True, bins=len(set(degrees)))  # 'kde=True' adds a density curve

  plt.tick_params(axis='both', labelsize=14)
  # Adding labels and title for clarity
  plt.xlabel('Degree', fontsize=14)
  plt.ylabel('Frequency', fontsize=14)

  plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def weighted_degrees_distribution(W,N):
  # Convert numpy array to a list and save it into 'degrees'
  ks = weighted_degrees_unsorted(W,N)

  # Draw the distribution plot
  sns.histplot(ks, kde=True, bins=16)  # 'kde=True' adds a density curve

  plt.tick_params(axis='both', labelsize=14)
  # Adding labels and title for clarity
  plt.xlabel('Weighted Degree', fontsize=14)
  plt.ylabel('Frequency', fontsize=14)

  plt.show()

In [ ]:
import numpy as np

def weighted_degrees(W, N):
    ks = []  # Start with an empty list to store the weighted degrees
    for i in range(N):  # Ensure the loop iterates over the range of neuron indices
        k_hat_i = np.sum(W[i, :])  # Calculate the weighted degree for neuron i
        ks.append(k_hat_i)  # Append the calculated weighted degree to the list
        ks.sort()
    return np.array(ks)  # Convert the list of weighted degrees to a numpy array and return it


In [ ]:
def weighted_degrees_unsorted(W, N):
    ks = []  # Start with an empty list to store the weighted degrees
    for i in range(N):  # Ensure the loop iterates over the range of neuron indices
        k_hat_i = np.sum(W[i, :])  # Calculate the weighted degree for neuron i
        ks.append(k_hat_i)  # Append the calculated weighted degree to the list

    return np.array(ks)  # Convert the list of weighted degrees to a numpy array and return it


In [ ]:
import matplotlib.pyplot as plt

def plot_weighted_degrees(ks, x_label="Sorted Neuron Index"):
    # Assuming ks is a numpy array of weighted degrees for each neuron
    N = len(ks)  # Number of neurons

    # Create a figure and axis for the plot
    plt.figure(figsize=(10, 6))

    # Plot ks against neuron indices
    plt.plot(range(N), ks, marker='o', linestyle='-', color='b')


    plt.tick_params(axis='both', labelsize=14)
    # Add title and labels
    plt.xlabel(f'{x_label}', fontsize=14)
    plt.ylabel('Weighted Degree', fontsize=14)

    # Show the plot
    plt.show()

### Neuron specific investigation

In [ ]:
import numpy as np

def vec_of_specific_X_row(X, W, index=None):
    # Count nonzero entries in each column to get the degrees
    nonzero_counts = np.count_nonzero(W, axis=0)

    if index == None:
      # Find the index of the column with the maximum degree
      index = np.argmax(nonzero_counts)

    # Extract and return the specific X row and degree of neuron {index}
    return X[:, index], nonzero_counts[index]

In [ ]:
import numpy as np

def vec_of_spec_X_row_weighted(X,W, N, index=None):
    # this function is different from the vec_of_specific_X_row as it returns weighted degree instead of degree
    # Get sorted weighted degrees
    ks = weighted_degrees(W,N)

    if index == None:
      # Find the index of the column with the degree
      index = np.argmax(ks)

    # Extract and return the state row with the degree specified and weighted degree of
    return X[index, :], ks[index], index


In [ ]:
import numpy as np

def find_spikes(vector):
    # Initialize two empty lists to store spike and valley values
    spikes = []
    valleys = []

    # Iterate through the vector, skipping the first and last elements
    for i in range(1, len(vector) - 1):
        # Check if the current value is greater than its neighbors (spike)
        if vector[i] > vector[i - 1] and vector[i] > vector[i + 1]:
            spikes.append(vector[i])
        # Check if the current value is less than its neighbors (valley)
        elif vector[i] < vector[i - 1] and vector[i] < vector[i + 1]:
            spikes.append(vector[i])

    return spikes

In [ ]:
def print_neuron_degrees(W, N, neuron_indices):
    """
    Print the degrees of specified neurons based on the weighted adjacency matrix W.

    Parameters:
    - W: Weighted adjacency matrix (numpy array) of the network.
    - N: Total number of neurons.
    - neuron_indices: List of neuron indices for which to print the degrees.
    """

    ks = weighted_degrees(W, N)  # Assuming this function returns an array of degrees for each neuron

    # Ensure neuron_indices are within the range of available neurons
    valid_indices = [index for index in neuron_indices if index < N]

    print("Neuron Degrees:")
    for index in valid_indices:
        print(f"Neuron {index}: Degree {ks[index]}")


In [ ]:
def find_similar_weighted_degrees(W, N, wd, epsilon):
    # First, calculate the weighted degrees for all neurons
    ks = weighted_degrees_unsorted(W,N)
    # Get the weighted degree of the i-th neuron


    # Find indices of all neurons with weighted degrees similar to the i-th neuron within epsilon
    similar_indices = []
    for j, k in enumerate(ks):  # Loop through each index and weighted degree
        if abs(k - wd) <= epsilon:  # Check if the weighted degree is similar
            similar_indices.append(j)  # Add the index to the list


    # Return the indices and their corresponding weighted degrees
    similar_degrees = []
    for j in similar_indices:  # Loop through each index of similar neurons
        similar_degrees.append(ks[j])  # Add the weighted degree to the list

    return similar_indices, similar_degrees

In [ ]:
import numpy as np

def find_weighted_degrees_around_value(W, N, target_value, epsilon):
    # Calculate the weighted degrees for all neurons
    ks = weighted_degrees(W,N)

    # Find indices of all neurons with weighted degrees within epsilon of the target value
    similar_indices = [j for j, k in enumerate(ks) if abs(k - target_value) <= epsilon]

    # Retrieve the corresponding weighted degrees for these indices
    similar_degrees = [ks[j] for j in similar_indices]

    return similar_indices, similar_degrees

### Plot specific neurons

In [ ]:
import matplotlib.pyplot as plt

def plot_X_vec(X_vec, ks, index):
    # Assuming X_vec is a 1D numpy array with the state of the neuron over time
    plt.figure(figsize=(10, 6))  # Set the figure size for better visibility

    # Plot the state vector over time
    plt.plot(X_vec, marker='o', linestyle='-', color='b')

    # Add title and labels to make the plot informative
    plt.title(f'State of Neuron {index} which has weighted degree={ks}')
    plt.xlabel('Time Step')
    plt.ylabel('State Value')

    plt.grid(True)  # Add a grid for better readability
    plt.show()  # Display the plot


In [ ]:
import matplotlib.pyplot as plt
import numpy as np  # Ensure NumPy is imported for indexing

def plot_X_similar_vec(X, similar_indices, connect_dots=False, start_time=None, end_time=None, legend=False, W=None, N=None):
    # Set the figure size for better visibility
    plt.figure(figsize=(10, 6))

    # Determine the time range for plotting
    if start_time is None:
        start_time = 0  # Start from the first time step if not specified
    if end_time is None or end_time > X.shape[1]:
        end_time = X.shape[1]  # End at the last time step if not specified or out of bounds

    # Colors for different lines
    colors = plt.cm.jet(np.linspace(0, 1, len(similar_indices)))
    if N is not None:
        ks = weighted_degrees_unsorted(W,N)
        nonzero_counts = np.count_nonzero(W, axis=1)

    # Loop over similar indices and plot each with a unique color
    for ind, color in zip(similar_indices, colors):
        time_range = range(start_time, end_time)
        # Plot the state vector for each similar index within the specified time range
        if connect_dots:
            plt.plot(time_range, X[ind, start_time:end_time], color=color, alpha=0.7)  # Connecting dots with lines

        if N is not None:
            plt.scatter(time_range, X[ind, start_time:end_time], color=color, alpha=0.5, s=10, label=f'Neuron {ind} (k={ks[ind]:.3f}, d={nonzero_counts[ind]})')  # Plotting the dots
        elif N is None:
          plt.scatter(time_range, X[ind, start_time:end_time], color=color, alpha=0.5, s=10, label=f'Neuron {ind}')

    plt.tick_params(axis='both', labelsize=15)
    plt.xlabel('Time Step', fontsize=15)
    plt.ylabel('State Value', fontsize=15)


    if legend:
        plt.legend(fontsize=15,loc="upper right")  # Add a legend to identify each neuron if lines are drawn

    plt.show()  # Display the plot




In [ ]:
# Function for full plot and top right zoomed in
import matplotlib.pyplot as plt
import numpy as np  # Ensure NumPy is imported for indexing
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def plot_X_similar_vec_2(X, similar_indices, zoom_start=None, zoom_end=None):
    # Set the figure size for better visibility
    fig, ax = plt.subplots(figsize=(10, 6))

    # Colors for different lines
    colors = plt.cm.jet(np.linspace(0, 1, len(similar_indices)))

    # Loop over similar indices and plot each with a unique color on the main plot
    for ind, color in zip(similar_indices, colors):
        ax.scatter(range(X.shape[1]), X[ind, :], color=color, alpha=0.5, s=10, label=f'Neuron {ind}')

    # Add title and labels to make the plot informative
    ax.set_title('States of Similar Neurons Over Time')
    ax.set_xlabel('Time Step')
    ax.set_ylabel('State Value')
    ax.grid(True)  # Add a grid for better readability
    ax.legend()  # Add a legend to identify each neuron

    # If zoom range is specified, create an inset (zoomed) plot within the main plot
    if zoom_start is not None and zoom_end is not None:
        # Define the inset axes' size and position [x, y, width, height] relative to the main axes
        axins = inset_axes(ax, width="30%", height="30%", loc='upper right')

        # Plot the same data on the inset axes, focusing on the specified zoom range
        for ind, color in zip(similar_indices, colors):
            axins.scatter(range(zoom_start, zoom_end), X[ind, zoom_start:zoom_end], color=color, alpha=0.5, s=10)

        # Set the zoomed plot's axis limits to the zoom range
        axins.set_xlim(zoom_start, zoom_end - 1)
        # Optionally, set the y-axis limits for the zoomed plot to focus on specific value ranges
        # axins.set_ylim(ymin, ymax)

        axins.grid(True)  # Add a grid for better readability

    plt.show()  # Display the plot


### Bifurcation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def run_network_and_plot_bifurcation(compute_state_matrix, F, vec_of_spec_X_row_weighted, find_spikes, param_values, index):
    # Setup for bifurcation diagram plotting

    param_list = []
    spike_values_list = []
    print("Starting the network execution and bifurcation analysis...")
    for i, alpha_value in enumerate(param_values):
        print(f"Processing alpha value {alpha_value:.3f} ({i+1}/{len(param_values)})...")

        # Now record the following T=300 for bifurcation analysis
        X, N, epsilon, rho, T, alpha, W = compute_state_matrix(100, 0.2, 0.8, 1300, alpha_value, F)

        # Get states of 95th neuron after transient period
        X_vec, ks, index = vec_of_spec_X_row_weighted(X[:,1000:], W, N, index)

        # Find spikes in the state vector of the 95th neuron
        spikes = find_spikes(X_vec)

        # Store the parameter value and corresponding spike values
        for spike_value in spikes:
            param_list.append(alpha_value)
            spike_values_list.append(spike_value)

    # Plotting the bifurcation diagram with larger bubbles
    plt.figure(figsize=(10, 6))
    plt.scatter(param_list, spike_values_list, s=10, c='k', alpha=0.5)  # Increase 's' for larger bubbles
    plt.title('Bifurcation Diagram')
    plt.xlabel('Control Parameter')
    plt.ylabel('Spike Values')
    plt.grid(True)
    plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def run_network_and_plot_bifurcation_2(compute_state_matrix, F, vec_of_spec_X_row_weighted, find_spikes, param_values, indices):
    # Setup for bifurcation diagram plotting
    plt.figure(figsize=(10, 6))
    colors = plt.cm.viridis(np.linspace(0, 1, len(indices)))  # Use a colormap to get different colors for each index

    print("Starting the network execution and bifurcation analysis...")
    for ind, color in zip(indices, colors):
        param_list = []
        spike_values_list = []
        for i, alpha_value in enumerate(param_values):
            print(f"Processing alpha value {alpha_value:.3f} for neuron index {ind} ({i+1}/{len(param_values)})...")

            # Run the network and record the following T=300 for bifurcation analysis
            X, N, epsilon, rho, T, alpha, W = compute_state_matrix(100, 0.2, 0.8, 1300, alpha_value, F)

            # Get states of the neuron at the specified index after transient period
            X_vec, ks, _ = vec_of_spec_X_row_weighted(X[:, 1000:], W, N, ind)

            # Find spikes in the state vector of the specified neuron
            spikes = find_spikes(X_vec)

            # Store the parameter value and corresponding spike values
            for spike_value in spikes:
                param_list.append(alpha_value)
                spike_values_list.append(spike_value)

        # Plotting the bifurcation diagram for the current neuron index
        plt.scatter(param_list, spike_values_list, s=10, c=[color], alpha=0.5, label=f'Neuron {ind}')

    plt.title('Bifurcation Diagrams')
    plt.xlabel('Control Parameter')
    plt.ylabel('Spike Values')
    plt.grid(True)
    plt.legend()
    plt.show()
    print("Bifurcation analysis completed.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def bifurcations_wrt_alpha(F, conn, N, epsilon, rho, param_values,
                                     T_transient, T, index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes):
    # Setup for bifurcation diagram plotting

    param_list = []
    spike_values_list = []
    print("Starting the network execution and bifurcation analysis...")
    for i, alpha_value in enumerate(param_values):
        print(f"Processing alpha value {alpha_value:.3f} ({i+1}/{len(param_values)})...")

        # Now record the following T=300 for bifurcation analysis
        X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                N, epsilon, rho, alpha_value, T,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)
        # Get states of 95th neuron after transient period
        X_vec, ks, index = vec_of_spec_X_row_weighted(X[:,T_transient:], W, N, index)

        # Find spikes in the state vector of the 95th neuron
        spikes = find_spikes(X_vec)

        # Store the parameter value and corresponding spike values
        for spike_value in spikes:
            param_list.append(alpha_value)
            spike_values_list.append(spike_value)

    return param_list, spike_values_list

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def bifurcations_wrt_rho(F, conn, N, epsilon, param_values, alpha,
                                     T_transient, T, index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes):
    # Setup for bifurcation diagram plotting

    param_list = []
    spike_values_list = []
    print("Starting the network execution and bifurcation analysis...")
    for i, rho_value in enumerate(param_values):
        print(f"Processing rho value {rho_value:.3f} ({i+1}/{len(param_values)})...")

        # Now record the following T=300 for bifurcation analysis
        X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                N, epsilon, rho_value, alpha, T,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)
        # Get states of 95th neuron after transient period
        X_vec, ks, index = vec_of_spec_X_row_weighted(X[:,T_transient:], W, N, index)

        # Find spikes in the state vector of the 95th neuron
        spikes = find_spikes(X_vec)

        # Store the parameter value and corresponding spike values
        for spike_value in spikes:
            param_list.append(rho_value)
            spike_values_list.append(spike_value)

    return param_list, spike_values_list

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def bifurcations_wrt_alpha_2(F, conn, N, epsilon, rho, param_values,
                                     T_transient, T, indices,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes):
    # Setup for bifurcation diagram plotting
    plt.figure(figsize=(10, 6))
    colors = plt.cm.viridis(np.linspace(0, 1, len(indices)))  # Use a colormap to get different colors for each index
    W = initialize_reservoir(N, epsilon, rho, ex_time=False)
    ks = weighted_degrees_unsorted(W,N)
    nonzero_counts = np.count_nonzero(W, axis=1)
    print("Starting the network execution and bifurcation analysis...")
    for ind, color in zip(indices, colors):
        param_list = []
        spike_values_list = []
        for i, alpha_value in enumerate(param_values):
            print(f"Processing alpha value {alpha_value:.3f} for neuron index {ind} ({i+1}/{len(param_values)})...")

            # Run the network and record the following T=300 for bifurcation analysis
            X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                N, epsilon, rho, alpha_value, T,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

            # Get states of the neuron at the specified index after transient period
            X_vec, _, _ = vec_of_spec_X_row_weighted(X[:, 1000:], W, N, ind)

            # Find spikes in the state vector of the specified neuron
            spikes = find_spikes(X_vec)

            # Store the parameter value and corresponding spike values
            for spike_value in spikes:
                param_list.append(alpha_value)
                spike_values_list.append(spike_value)

        # Plotting the bifurcation diagram for the current neuron index
        plt.scatter(param_list, spike_values_list, s=10, c=[color], alpha=0.5, label=f'Neuron {ind} (k={ks[ind]:.3f}, d={nonzero_counts[ind]})')
    plt.tick_params(axis='both', labelsize=15)
    plt.ylabel('Spike Values',fontsize=15)
    plt.legend(fontsize=15, loc="upper right")
    plt.show()
    print("Bifurcation analysis completed.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def bifurcations_wrt_rho_2(F, conn, N, epsilon, param_values, alpha,
                                     T_transient, T, indices,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes):
    # Setup for bifurcation diagram plotting
    plt.figure(figsize=(10, 6))
    colors = plt.cm.viridis(np.linspace(0, 1, len(indices)))  # Use a colormap to get different colors for each index
    W = initialize_reservoir(N, epsilon, 0.8, ex_time=False)
    ks = weighted_degrees_unsorted(W,N)
    nonzero_counts = np.count_nonzero(W, axis=1)

    print("Starting the network execution and bifurcation analysis...")
    for ind, color in zip(indices, colors):
        param_list = []
        spike_values_list = []
        for i, rho_value in enumerate(param_values):
            print(f"Processing alpha value {rho_value:.3f} for neuron index {ind} ({i+1}/{len(param_values)})...")

            # Run the network and record the following T=300 for bifurcation analysis
            X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                N, epsilon, rho_value, alpha, T,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

            # Get states of the neuron at the specified index after transient period
            X_vec, _, _ = vec_of_spec_X_row_weighted(X[:, T_transient:], W, N, ind)

            # Find spikes in the state vector of the specified neuron
            spikes = find_spikes(X_vec)

            # Store the parameter value and corresponding spike values
            for spike_value in spikes:
                param_list.append(rho_value)
                spike_values_list.append(spike_value)

        # Plotting the bifurcation diagram for the current neuron index
        plt.scatter(param_list, spike_values_list, s=10, c=[color], alpha=0.5, label=f'Neuron {ind} (k={ks[ind]:.3f}, d={nonzero_counts[ind]})')
    plt.tick_params(axis='both', labelsize=15)
    plt.ylabel('Spike Values',fontsize=15)
    plt.legend(fontsize=15, loc="upper right")
    plt.show()
    print("Bifurcation analysis completed.")

In [ ]:
def plot_bifurcation(param_list, spike_values_list):
    # Plotting the bifurcation diagram with larger bubbles
    plt.figure(figsize=(10, 6))
    plt.scatter(param_list, spike_values_list, s=10, c='k', alpha=0.5)  # Increase 's' for larger bubbles
    plt.tick_params(axis='both', labelsize=15)
    plt.ylabel('Spike Values', fontsize=15)
    plt.show()

### Laplacian matrix

In [ ]:
import numpy as np

def calculate_laplacian(W):

    # Degree matrix D
    k_hat_vec = weighted_degrees_unsorted(W, W.shape[0])
    D = np.diag(k_hat_vec)

    # Laplacian matrix L = D - A
    L = D - W

    return L


## Degree of neurons

Degrees of neuron is number of neurons it is connected to.
ie if degree = 1 it is connected only with neuron other or itself
degree = 2 it is connected with 2 neurons

In [ ]:
def degrees_distribution(W):
  # Count nonzero entries in each column
  nonzero_counts = np.count_nonzero(W, axis=0)

  # Convert numpy array to a list and save it into 'degrees'
  degrees = nonzero_counts.tolist()

  import seaborn as sns
  import matplotlib.pyplot as plt
  # Draw the distribution plot
  sns.histplot(degrees, kde=True, bins=len(set(degrees)))  # 'kde=True' adds a density curve

  # Adding labels and title for clarity
  plt.xlabel('Degree')
  plt.ylabel('Frequency')
  plt.title('Distribution of Neuron Degrees')

  plt.show()

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(100, 0.2, 0.8 , 300, 0.01, F=F_mod)

In [ ]:
degrees_distribution(W)

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(100, 0.4, 0.8 , 300, 0.01, F=F_mod)

In [ ]:
degrees_distribution(W)

## Weighted Degrees

In [ ]:
import numpy as np

def weighted_degrees(W, N):
    ks = []  # Start with an empty list to store the weighted degrees
    for i in range(N):  # Ensure the loop iterates over the range of neuron indices
        k_hat_i = np.sum(W[i, :])  # Calculate the weighted degree for neuron i
        ks.append(k_hat_i)  # Append the calculated weighted degree to the list
        ks.sort()
    return np.array(ks)  # Convert the list of weighted degrees to a numpy array and return it



In [ ]:
import matplotlib.pyplot as plt

def plot_weighted_degrees(ks):
    # Assuming ks is a numpy array of weighted degrees for each neuron
    N = len(ks)  # Number of neurons

    # Create a figure and axis for the plot
    plt.figure(figsize=(10, 6))

    # Plot ks against neuron indices
    plt.plot(range(N), ks, marker='o', linestyle='-', color='b')

    # Add title and labels
    plt.title('Weighted Degrees of Neurons')
    plt.xlabel('Neuron Index')
    plt.ylabel('Weighted Degree')

    # Optionally, add a grid for better readability
    plt.grid(True)

    # Show the plot
    plt.show()


In [ ]:
plot_weighted_degrees(weighted_degrees(W,N))

## Bifurcation diagram

In [ ]:
import numpy as np

def max_degree_vector(X, W, index=None):
    # Count nonzero entries in each column to get the degrees
    nonzero_counts = np.count_nonzero(W, axis=0)

    if index == None:
      # Find the index of the column with the maximum degree
      index = np.argmax(nonzero_counts)

    # Extract and return the column with the highest degree
    return X[:, index], nonzero_counts[index]



In [ ]:
import numpy as np

def max_weighted_degree(X,W, N, index=None):

    ks = weighted_degrees(W,N)

    if index == None:
      # Find the index of the column with the maximum degree
      index = np.argmax(ks)

    # Extract and return the state row with the degree specified and weighted degree of it
    return X[:, index], ks[index]



In [ ]:
import numpy as np

def find_spikes(vector):
    # Initialize an empty list to store spike values
    spikes = []

    # Iterate through the vector, skipping the first and last elements
    for i in range(1, len(vector) - 1):
        # Check if the current value is greater than its neighbors
        if vector[i] > vector[i - 1] and vector[i] > vector[i + 1]:
            spikes.append(vector[i])

    return spikes


### State Network

In [ ]:
import matplotlib.pyplot as plt

# Setup for bifurcation diagram plotting
param_values = np.linspace(0, 0.3, 10)  # Control parameter range
param_list = []
spike_values_list = []

for alpha_value in param_values:
    # Assume 'network' function returns W among other outputs
    X, N, epsilon, rho, T, alpha, W= compute_state_matrix(100, 0.4, 0.8 , 300, alpha_value, F=F_mod)

    # Get the vector with the highest degree
    max_d_vec,_ = max_weighted_degree(X, W, N)

    # Find spikes in the max_degree_vector
    spikes = find_spikes(max_d_vec)

    # Store the parameter value and corresponding spike values
    for spike_value in spikes:
        param_list.append(alpha_value)
        spike_values_list.append(spike_value)


# Plotting the bifurcation diagram with larger bubbles
plt.figure(figsize=(10, 6))
plt.scatter(param_list, spike_values_list, s=25, c='k', alpha=0.5)  # Increase 's' for larger bubbles
plt.title('Bifurcation Diagram')
plt.xlabel('Control Parameter')
plt.ylabel('Spike Values')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Setup for bifurcation diagram plotting
param_values = np.linspace(0, 0.3, 30)  # Control parameter range
param_list = []
spike_values_list = []

for alpha_value in param_values:
    # Assume 'network' function returns W among other outputs
    X, N, epsilon, rho, T, alpha, W= compute_state_matrix(100, 0.2, 0.8 , 300, alpha_value, F=F_mod)

    # Get the vector with the highest degree
    max_d_vec,_ = max_weighted_degree(X, W, N)

    # Find spikes in the max_degree_vector
    spikes = find_spikes(max_d_vec)

    # Store the parameter value and corresponding spike values
    for spike_value in spikes:
        param_list.append(alpha_value)
        spike_values_list.append(spike_value)


# Plotting the bifurcation diagram with larger bubbles
plt.figure(figsize=(10, 6))
plt.scatter(param_list, spike_values_list, s=25, c='k', alpha=0.5)  # Increase 's' for larger bubbles
plt.title('Bifurcation Diagram')
plt.xlabel('Control Parameter')
plt.ylabel('Spike Values')
plt.grid(True)
plt.show()

## Similar Weighted Degrees

In [ ]:
def find_similar_weighted_degrees(W, N, i, epsilon):
    # First, calculate the weighted degrees for all neurons
    ks = [np.sum(W[j, :]) for j in range(N)]
    # Get the weighted degree of the i-th neuron
    k_i = ks[i]

    # Find indices of all neurons with weighted degrees similar to the i-th neuron within epsilon
    similar_indices = []
    for j, k in enumerate(ks):  # Loop through each index and weighted degree
        if abs(k - k_i) <= epsilon:  # Check if the weighted degree is similar
            similar_indices.append(j)  # Add the index to the list


    # Return the indices and their corresponding weighted degrees
    similar_degrees = []
    for j in similar_indices:  # Loop through each index of similar neurons
        similar_degrees.append(ks[j])  # Add the weighted degree to the list

    return similar_indices, similar_degrees

In [ ]:
import numpy as np

def find_weighted_degrees_around_value(W, N, target_value, epsilon):
    # Calculate the weighted degrees for all neurons
    ks = [np.sum(W[j, :]) for j in range(N)]

    # Find indices of all neurons with weighted degrees within epsilon of the target value
    similar_indices = [j for j, k in enumerate(ks) if abs(k - target_value) <= epsilon]

    # Retrieve the corresponding weighted degrees for these indices
    similar_degrees = [ks[j] for j in similar_indices]

    return similar_indices, similar_degrees


In [ ]:
import matplotlib.pyplot as plt

def plot_X_max_d_vec(X_max_d_vec, ks):
    # Assuming X_max_d_vec is a 1D numpy array with the state of the neuron over time
    plt.figure(figsize=(10, 6))  # Set the figure size for better visibility

    # Plot the state vector over time
    plt.plot(X_max_d_vec, marker='o', linestyle='-', color='b')

    # Add title and labels to make the plot informative
    plt.title(f'State of Neuron which has weighted degree={ks}')
    plt.xlabel('Time Step')
    plt.ylabel('State Value')

    plt.grid(True)  # Add a grid for better readability
    plt.show()  # Display the plot


In [ ]:
    X, N, epsilon, rho, T, alpha, W= compute_state_matrix(100, 0.2, 0.8 , 300, 0.08, F=F_mod)

    # Get the vector with the highest degree
    X_max_d_vec,_ = max_weighted_degree(X, W, N)

    plot_X_max_d_vec(X_max_d_vec)



In [ ]:
    X, N, epsilon, rho, T, alpha, W= compute_state_matrix(100, 0.2, 0.8 , 300, 0.04, F=F_mod)

    # Get the vector with the highest degree
    X_max_d_vec,_ = max_weighted_degree(X, W, N)

    plot_X_max_d_vec(X_max_d_vec)

In [ ]:
X, N, epsilon, rho, T, alpha, W= compute_state_matrix(100, 0.2, 0.8 , 300, 0.08, F=F_mod)

In [ ]:
plot_weighted_degrees(weighted_degrees(W,N))

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 95, 0.005)

In [ ]:
print(similar_indices)
print(similar_degrees)

Actually finding similar degrees of sorted array doesnt allow me to find its state vector X as indice will be different. Thus we use target value from now on

In [ ]:
similar_indices, similar_degrees = find_weighted_degrees_around_value(W, N, 1.12, 0.005)

In [ ]:
print(similar_indices)
print(similar_degrees)

In [ ]:
for i in similar_indices:
  plot_X_max_d_vec(X[i,:])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np  # Ensure NumPy is imported for indexing

def plot_X_of_vec(X, similar_indices):
    # Set the figure size for better visibility
    plt.figure(figsize=(10, 6))

    # Colors for different lines
    colors = plt.cm.jet(np.linspace(0, 1, len(similar_indices)))

    # Loop over similar indices and plot each with a unique color
    for ind, color in zip(similar_indices, colors):
        # Plot the state vector for each similar index with bubbles only
        plt.scatter(range(X.shape[1]), X[ind, :], color=color, alpha=0.5, label=f'Neuron {ind}')

    # Add title and labels to make the plot informative
    plt.title('States of Similar Neurons Over Time')
    plt.xlabel('Time Step')
    plt.ylabel('State Value')

    plt.grid(True)  # Add a grid for better readability
    plt.legend()  # Add a legend to identify each neuron
    plt.show()  # Display the plot


In [ ]:
plot_X_of_vec(X,similar_indices)

### Neurons of alpha=0.08 state matrix

In [ ]:
X, N, epsilon, rho, T, alpha, W= compute_state_matrix(100, 0.2, 0.8 , 1000, 0.08, F=F_mod)
plot_weighted_degrees(weighted_degrees(W,N))


In [ ]:
X_max_d_vec,weighted_degree = max_weighted_degree(X, W, N, 2)
plot_X_max_d_vec(X_max_d_vec, weighted_degree)

In [ ]:
X_max_d_vec,weighted_degree = max_weighted_degree(X, W, N, 98)
plot_X_max_d_vec(X_max_d_vec, weighted_degree)

In [ ]:
X_max_d_vec,weighted_degree = max_weighted_degree(X, W, N, 97)
plot_X_max_d_vec(X_max_d_vec, weighted_degree)

In [ ]:
X_max_d_vec,weighted_degree = max_weighted_degree(X, W, N, 96)
plot_X_max_d_vec(X_max_d_vec, weighted_degree)

In [ ]:
X_max_d_vec,weighted_degree = max_weighted_degree(X, W, N, 95)
plot_X_max_d_vec(X_max_d_vec, weighted_degree)

In [ ]:
similar_indices, similar_degrees = find_weighted_degrees_around_value(W, N, 1.12, 0.005)

In [ ]:
print(similar_indices)

In [ ]:
for i, ks in (similar_indices,similar_degrees):
  plot_X_max_d_vec(X[i,:], ks)
plot_X_of_vec(X,similar_indices)

## All the functions in this part up to this point

In [ ]:
def degrees_distribution(W):
  # Count nonzero entries in each column
  nonzero_counts = np.count_nonzero(W, axis=0)

  # Convert numpy array to a list and save it into 'degrees'
  degrees = nonzero_counts.tolist()

  import seaborn as sns
  import matplotlib.pyplot as plt
  # Draw the distribution plot
  sns.histplot(degrees, kde=True, bins=len(set(degrees)))  # 'kde=True' adds a density curve

  # Adding labels and title for clarity
  plt.xlabel('Degree')
  plt.ylabel('Frequency')
  plt.title('Distribution of Neuron Degrees')

  plt.show()

In [ ]:
import numpy as np

def weighted_degrees(W, N):
    ks = []  # Start with an empty list to store the weighted degrees
    for i in range(N):  # Ensure the loop iterates over the range of neuron indices
        k_hat_i = np.sum(W[i, :])  # Calculate the weighted degree for neuron i
        ks.append(k_hat_i)  # Append the calculated weighted degree to the list
        ks.sort()
    return np.array(ks)  # Convert the list of weighted degrees to a numpy array and return it


In [ ]:
import matplotlib.pyplot as plt

def plot_weighted_degrees(ks):
    # Assuming ks is a numpy array of weighted degrees for each neuron
    N = len(ks)  # Number of neurons

    # Create a figure and axis for the plot
    plt.figure(figsize=(10, 6))

    # Plot ks against neuron indices
    plt.plot(range(N), ks, marker='o', linestyle='-', color='b')

    # Add title and labels
    plt.title('Weighted Degrees of Neurons')
    plt.xlabel('Neuron Index')
    plt.ylabel('Weighted Degree')

    # Optionally, add a grid for better readability
    plt.grid(True)

    # Show the plot
    plt.show()

In [ ]:
import numpy as np

def vec_of_specific_X_row(X, W, index=None):
    # Count nonzero entries in each column to get the degrees
    nonzero_counts = np.count_nonzero(W, axis=0)

    if index == None:
      # Find the index of the column with the maximum degree
      index = np.argmax(nonzero_counts)

    # Extract and return the specific X row and degree of neuron {index}
    return X[:, index], nonzero_counts[index]

In [ ]:
import numpy as np

def vec_of_spec_X_row_weighted(X,W, N, index=None):
    # this function is different from the vec_of_specific_X_row as it returns weighted degree instead of degree
    # Get sorted weighted degrees
    ks = weighted_degrees(W,N)

    if index == None:
      # Find the index of the column with the degree
      index = np.argmax(ks)

    # Extract and return the state row with the degree specified and weighted degree of
    return X[index, :], ks[index], index


In [ ]:
import numpy as np

def find_spikes(vector):
    # Initialize an empty list to store spike values
    spikes = []

    # Iterate through the vector, skipping the first and last elements
    for i in range(1, len(vector) - 1):
        # Check if the current value is greater than its neighbors
        if vector[i] > vector[i - 1] and vector[i] > vector[i + 1]:
            spikes.append(vector[i])

    return spikes


In [ ]:
def find_similar_weighted_degrees(W, N, i, epsilon):
    # First, calculate the weighted degrees for all neurons
    ks = weighted_degrees(W,N)
    # Get the weighted degree of the i-th neuron
    k_i = ks[i]

    # Find indices of all neurons with weighted degrees similar to the i-th neuron within epsilon
    similar_indices = []
    for j, k in enumerate(ks):  # Loop through each index and weighted degree
        if abs(k - k_i) <= epsilon:  # Check if the weighted degree is similar
            similar_indices.append(j)  # Add the index to the list


    # Return the indices and their corresponding weighted degrees
    similar_degrees = []
    for j in similar_indices:  # Loop through each index of similar neurons
        similar_degrees.append(ks[j])  # Add the weighted degree to the list

    return similar_indices, similar_degrees

In [ ]:
import numpy as np

def find_weighted_degrees_around_value(W, N, target_value, epsilon):
    # Calculate the weighted degrees for all neurons
    ks = weighted_degrees(W,N)

    # Find indices of all neurons with weighted degrees within epsilon of the target value
    similar_indices = [j for j, k in enumerate(ks) if abs(k - target_value) <= epsilon]

    # Retrieve the corresponding weighted degrees for these indices
    similar_degrees = [ks[j] for j in similar_indices]

    return similar_indices, similar_degrees

In [ ]:
import matplotlib.pyplot as plt

def plot_X_vec(X_vec, ks, index):
    # Assuming X_vec is a 1D numpy array with the state of the neuron over time
    plt.figure(figsize=(10, 6))  # Set the figure size for better visibility

    # Plot the state vector over time
    plt.plot(X_vec, marker='o', linestyle='-', color='b')

    # Add title and labels to make the plot informative
    plt.title(f'State of Neuron {index} which has weighted degree={ks}')
    plt.xlabel('Time Step')
    plt.ylabel('State Value')

    plt.grid(True)  # Add a grid for better readability
    plt.show()  # Display the plot


In [ ]:
import matplotlib.pyplot as plt
import numpy as np  # Ensure NumPy is imported for indexing

def plot_X_simillar_vec(X, similar_indices):

    # Set the figure size for better visibility
    plt.figure(figsize=(10, 6))

    # Colors for different lines
    colors = plt.cm.jet(np.linspace(0, 1, len(similar_indices)))

    # Loop over similar indices and plot each with a unique color
    for ind, color in zip(similar_indices, colors):
        # Plot the state vector for each similar index with bubbles only
        plt.scatter(range(X.shape[1]), X[ind, :], color=color, alpha=0.5, label=f'Neuron {ind}')

    # Add title and labels to make the plot informative
    plt.title('States of Similar Neurons Over Time')
    plt.xlabel('Time Step')
    plt.ylabel('State Value')

    plt.grid(True)  # Add a grid for better readability
    plt.legend()  # Add a legend to identify each neuron
    plt.show()  # Display the plot


# Reservoir Analysis

## Hyperparameters investigation of network with F=F_identity



In [ ]:
F=F_identity

### Chemical Network

In [ ]:
conn="Chemical"

#### Reservoirs with N=10, Converging diagrams

##### Reservoir 0, rho=0.5, alpha=0.05

In [ ]:
# def compute_state_matrix(F, connectivity, N, epsilon, rho, alpha, T,
                         # Lapl=True, ex_time=False):
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 0.5, 0.05, 100)

In [ ]:

computational_units_state(X, True)

##### Reservoir 1 rho=0.5, alpha=0.5



In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 0.5, 0.5, 30)

In [ ]:
computational_units_state(X, True)

##### Reservoir 2 rho=1.5, alpha=0.05

In [ ]:
# Run the Network with N = 100, epsilon=0.2, rho = 0.95, T = 1000, alpha = 1.1
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 1.5, 0.05, 300)


In [ ]:
computational_units_state(X,True)

In [ ]:
states_average_plot(X)

##### Reservoir 3 rho=1.5, alpha=*0.5*

In [ ]:
# Run the Network with N = 100, epsilon=0.2, rho = 1.1, T = 1000, alpha = 0.05
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 1.5, 0.5, 50)

In [ ]:
computational_units_state(X, True)

##### Reservoir 4 rho=0.95, alpha=0.05

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 0.95, 0.05, 100)

In [ ]:
computational_units_state(X, True)

#### Reservoirs with N=100

##### Reservoir 0, rho=0.5, alpha=0.05

In [ ]:
# def compute_state_matrix(F, connectivity, N, epsilon, rho, alpha, T,
                         # Lapl=True, ex_time=False):
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.5, 0.05, 100)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=False)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

In [ ]:
degrees_distribution(W)
plot_weighted_degrees(weighted_degrees(W,N))

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 70, 0.01)
plot_X_similar_vec(X, similar_indices)

##### Reservoir 1 rho=0.5, alpha=0.5



In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.5, 0.5, 30)


In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

In [ ]:
degrees_distribution(W)
plot_weighted_degrees(weighted_degrees(W,N))

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 70, 0.01)
plot_X_similar_vec(X, similar_indices)

##### Reservoir 2 rho=1.5, alpha=0.05

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.05, 1000)


In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=False)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

In [ ]:
pear_graph(X, future_time=1000)

In [ ]:
degrees_distribution(W)
plot_weighted_degrees(weighted_degrees(W,N))

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 70, 0.01)
plot_X_similar_vec(X, similar_indices)

##### Reservoir 3 rho=1.5, alpha=*0.5*

In [ ]:
# Run the Network with N = 100, epsilon=0.2, rho = 1.1, T = 1000, alpha = 0.05
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.5, 1000)

In [ ]:
computational_units_state(X)

In [ ]:
#def computational_units_state_2(X, nn, t_start, t_end, zoom_in_start=None, zoom_in_end=None, legend=False):

In [ ]:
computational_units_state_2(X, 10, 0, 1000,y_lim=[-1,1], zoom_in_start=900, zoom_in_end=1000, legend=True)

In [ ]:
# Plots
states_average_plot(X, running_avg=False)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

In [ ]:
pear_graph(X, future_time=1000)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 70, 0.01)
plot_X_similar_vec(X, similar_indices, True, 500,600)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 25, 0.01)
plot_X_similar_vec(X, similar_indices, True, 500,600)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 90, 0.01)
plot_X_similar_vec(X, similar_indices, True, 500,600)

##### Reservoir 4 rho=0.95, alpha=0.05

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.95, 0.05, 500)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

In [ ]:
pear_graph(X, future_time=500)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 70, 0.01)
plot_X_similar_vec(X, similar_indices, True)

##### Reservoir 5 rho=1.5, alpha=0.6

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.6, 1000)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 70, 0.01)
plot_X_similar_vec(X, similar_indices, True, 500,600)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 15, 0.01)
plot_X_similar_vec(X, similar_indices, True, 500,600)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 90, 0.01)
plot_X_similar_vec(X, similar_indices, True, 500,600)

##### Reservoir 6 rho=1.5, alpha=1.1

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 1.1, 1000)

In [ ]:
computational_units_state(X, y_lim=[-1.5,1.5])

In [ ]:
computational_units_state_2(X, 5, 0, 1000,y_lim=[-1.5,1.5], zoom_in_start=950, zoom_in_end=1000, legend=True)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 15, 0.01)
plot_X_similar_vec(X, similar_indices, True, 500,550)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 90, 0.01)
plot_X_similar_vec(X, similar_indices, True, 500,550)

##### Reservoir 7 rho=2, alpha=2

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 2, 2, 1000)

In [ ]:
computational_units_state(X, y_lim=[-4,4])
states_average_plot(X, running_avg=True)
states_average_plot_zoom_in(X, 250, 400, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

In [ ]:
computational_units_state_2(X, 5, 0, 1000,y_lim=[-1000,1000], zoom_in_start=200, zoom_in_end=250, legend=True)

##### Reservoir 8 rho=1.5, alpha=0.95

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F_identity, "Chemical", 100, 0.2, 1.5, 0.95, 1000)

In [ ]:
computational_units_state(X,False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 9 rho=2, alpha=0.95

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 2, 0.95, 1000)

In [ ]:
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 10 rho=2, alpha=0.8

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 2, 0.8, 1000)
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 11 rho=0.8, alpha=0.8

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.8, 0.8, 1000)
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 12 rho=0.8, alpha=0.01

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.8, 0.01, 1000)
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 13 rho=2, alpha=1.5

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.4, 1.5, 1000)

In [ ]:
computational_units_state(X, y_lim=[-4,4])
states_average_plot(X, running_avg=True)
states_average_plot_zoom_in(X, 250, 400, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

In [ ]:
computational_units_state_2(X, 5, 0, 1000,y_lim=[-1000,1000], zoom_in_start=200, zoom_in_end=250, legend=True)

### Electrical Network

In [ ]:
conn="Electrical"

#### Reservoirs with N=10, Converging diagrams

##### Reservoir 0, rho=0.5, alpha=0.05

In [ ]:
# def compute_state_matrix(F, connectivity, N, epsilon, rho, alpha, T,
                         # Lapl=True, ex_time=False):
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 0.5, 0.05, 100)

In [ ]:
computational_units_state(X, True)

##### Reservoir 1 rho=0.5, alpha=0.5



In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 0.5, 0.5, 30)

In [ ]:
computational_units_state(X, True)

##### Reservoir 2 rho=1.5, alpha=0.05

In [ ]:
# Run the Network with N = 100, epsilon=0.2, rho = 0.95, T = 1000, alpha = 1.1
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 1.5, 0.05, 300)


In [ ]:
computational_units_state(X,True)

##### Reservoir 3 rho=1.5, alpha=*0.5*

In [ ]:
# Run the Network with N = 100, epsilon=0.2, rho = 1.1, T = 1000, alpha = 0.05
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 1.5, 0.5, 50)

In [ ]:
computational_units_state(X, True)

##### Reservoir 4 rho=0.95, alpha=0.05

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 0.95, 0.05, 100)

In [ ]:
computational_units_state(X, True)

#### Reservoirs with N=100

##### Reservoir 0, rho=0.5, alpha=0.05

In [ ]:
# def compute_state_matrix(F, connectivity, N, epsilon, rho, alpha, T,
                         # Lapl=True, ex_time=False):
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.5, 0.05, 100)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 1 rho=0.5, alpha=0.5



In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.5, 0.5, 30)


In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 2 rho=1.5, alpha=0.05

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.05, 100)


In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 3 rho=1.5, alpha=*0.5*

In [ ]:
# Run the Network with N = 100, epsilon=0.2, rho = 1.1, T = 1000, alpha = 0.05
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.5, 500)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 4 rho=0.95, alpha=0.05

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.95, 0.05, 500)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 5 rho=1.5, alpha=0.6

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.6, 1000)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 6 rho=1.5, alpha=1.1

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 1.1, 1000)

In [ ]:
computational_units_state(X, y_lim=[-1.5,1.5])

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 7 rho=2, alpha=2

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 2, 2, 1000)

In [ ]:
computational_units_state(X, y_lim=[-4,4])
states_average_plot(X, running_avg=True)
states_average_plot_zoom_in(X, 250, 400, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 8 rho=1.5, alpha=0.95

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F_identity, "Electrical", 100, 0.2, 1.5, 0.95, 1000)

In [ ]:
computational_units_state(X,False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 9 rho=2, alpha=0.95

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 2, 0.95, 1000)

In [ ]:
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 10 rho=2, alpha=0.8

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 2, 0.8, 1000)
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 11 rho=0.8, alpha=0.8

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.8, 0.8, 1000)
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 12 rho=0.8, alpha=0.01

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.8, 0.01, 1000)
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

## Hyperparameters investigation of network with F=F_mod



In [ ]:
F=F_mod

### Chemical Network

In [ ]:
conn="Chemical"

#### Reservoirs with N=10, Converging diagrams

##### Reservoir 0, rho=0.5, alpha=0.05

In [ ]:
# def compute_state_matrix(F, connectivity, N, epsilon, rho, alpha, T,
                         # Lapl=True, ex_time=False):
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 0.5, 0.05, 100)

In [ ]:
computational_units_state(X, True)

##### Reservoir 1 rho=0.5, alpha=0.5



In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 0.5, 0.5, 30)

In [ ]:
computational_units_state(X, True)

##### Reservoir 2 rho=1.5, alpha=0.05

In [ ]:
# Run the Network with N = 100, epsilon=0.2, rho = 0.95, T = 1000, alpha = 1.1
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 1.5, 0.05, 300)


In [ ]:
computational_units_state(X,True)

##### Reservoir 3 rho=1.5, alpha=*0.5*

In [ ]:
# Run the Network with N = 100, epsilon=0.2, rho = 1.1, T = 1000, alpha = 0.05
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 1.5, 0.5, 50)

In [ ]:
computational_units_state(X, True)

##### Reservoir 4 rho=0.95, alpha=0.05

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 0.95, 0.05, 100)

In [ ]:
computational_units_state(X, True)

#### Reservoirs with N=100

##### Reservoir 0, rho=0.5, alpha=0.05

In [ ]:
# def compute_state_matrix(F, connectivity, N, epsilon, rho, alpha, T,
                         # Lapl=True, ex_time=False):
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.5, 0.05, 100)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 1 rho=0.5, alpha=0.5



In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.5, 0.5, 30)


In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 2 rho=1.5, alpha=0.05

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.05, 100)


In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 3 rho=1.5, alpha=*0.5*

In [ ]:
# Run the Network with N = 100, epsilon=0.2, rho = 1.1, T = 1000, alpha = 0.05
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.5, 500)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 4 rho=0.95, alpha=0.05

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.95, 0.05, 500)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 5 rho=1.5, alpha=0.6

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.6, 1000)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 6 rho=1.5, alpha=1.1

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 1.1, 1000)

In [ ]:
computational_units_state(X, y_lim=[-1.5,1.5])

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 7 rho=2, alpha=2

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 2, 2, 1000)

In [ ]:
computational_units_state(X, y_lim=[-4,4])
states_average_plot(X, running_avg=True)
states_average_plot_zoom_in(X, 250, 400, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 8 rho=1.5, alpha=0.95

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F_identity, "Chemical", 100, 0.2, 1.5, 0.95, 1000)

In [ ]:
computational_units_state(X,False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 9 rho=2, alpha=0.95

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 2, 0.95, 1000)

In [ ]:
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 10 rho=2, alpha=0.8

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 2, 0.8, 1000)
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 11 rho=0.8, alpha=0.8

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.8, 0.8, 1000)
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 12 rho=0.8, alpha=0.01

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.8, 0.01, 1000)
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

### Electrical Network

In [ ]:
conn="Electrical"

#### Reservoirs with N=10, Converging diagrams

##### Reservoir 0, rho=0.5, alpha=0.05

In [ ]:
# def compute_state_matrix(F, connectivity, N, epsilon, rho, alpha, T,
                         # Lapl=True, ex_time=False):
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 0.5, 0.05, 100)

In [ ]:
computational_units_state(X, True)

##### Reservoir 1 rho=0.5, alpha=0.5



In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 0.5, 0.5, 30)

In [ ]:
computational_units_state(X, True)

##### Reservoir 2 rho=1.5, alpha=0.05

In [ ]:
# Run the Network with N = 100, epsilon=0.2, rho = 0.95, T = 1000, alpha = 1.1
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 1.5, 0.05, 300)


In [ ]:
computational_units_state(X,True)

##### Reservoir 3 rho=1.5, alpha=*0.5*

In [ ]:
# Run the Network with N = 100, epsilon=0.2, rho = 1.1, T = 1000, alpha = 0.05
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 1.5, 0.5, 50)

In [ ]:
computational_units_state(X, True)

##### Reservoir 4 rho=0.95, alpha=0.05

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 10, 0.2, 0.95, 0.05, 100)

In [ ]:
computational_units_state(X, True)

#### Reservoirs with N=100

##### Reservoir 0, rho=0.5, alpha=0.05

In [ ]:
# def compute_state_matrix(F, connectivity, N, epsilon, rho, alpha, T,
                         # Lapl=True, ex_time=False):
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.5, 0.05, 100)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 1 rho=0.5, alpha=0.5



In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.5, 0.5, 30)


In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 2 rho=1.5, alpha=0.05

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.05, 100)


In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 3 rho=1.5, alpha=*0.5*

In [ ]:
# Run the Network with N = 100, epsilon=0.2, rho = 1.1, T = 1000, alpha = 0.05
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.5, 500)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 4 rho=0.95, alpha=0.05

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.95, 0.05, 500)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 5 rho=1.5, alpha=0.6

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.6, 1000)

In [ ]:
computational_units_state(X)

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 6 rho=1.5, alpha=1.1

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 1.1, 1000)

In [ ]:
computational_units_state(X, y_lim=[-1.5,1.5])

In [ ]:
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)

##### Reservoir 7 rho=2, alpha=2

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 2, 2, 1000)

In [ ]:
computational_units_state(X, y_lim=[-4,4])
states_average_plot(X, running_avg=True)
states_average_plot_zoom_in(X, 250, 400, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 8 rho=1.5, alpha=0.95

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 0.95, 1000)

In [ ]:
computational_units_state(X,False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 9 rho=2, alpha=0.95

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 2, 0.95, 1000)

In [ ]:
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 10 rho=2, alpha=0.8

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 2, 0.8, 1000)
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 11 rho=0.8, alpha=0.8

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.8, 0.8, 1000)
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

##### Reservoir 12 rho=0.8, alpha=0.01

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 0.8, 0.01, 1000)
computational_units_state(X, False)
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)

## Hyperparameters Chaos Analysis

### F=identity

In [ ]:
F=F_identity

#### Chemical

In [ ]:
conn = "Chemical"

##### Integral of Pearson Correlation

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 1.1, 1000)
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)


In [ ]:
area_sum, integral = pearson_integral(X)
pear_graph(X, integrals=integral)

In [ ]:
print(area_sum, integral)

##### Integral of FFT

In [ ]:
# integral_of_spikes(X, treshold=10, window_size=10, running_avg= False, integral_analysis=False, f_min=None, f_max=None):
area_sum, integrals, spike_count = integral_of_spikes(X, threshold = 5, integral_analysis=True)
print(area_sum, spike_count)

In [ ]:
fft_plot(X, threshold=5,integrals=integrals)

##### Example of *Grid*

In [ ]:
rho = [0.05, 0.25, 0.5, 0.8, 0.95, 1.1, 1.3, 1.5, 2, 2.5, 3]
alpha = [0, 0.00001, 0.01, 0.2, 0.05, 0.10, 0.15, 0.25, 0.5, 0.7, 0.8, 0.95, 1, 1.1, 1.5,2]

In [ ]:
S_test = grid(F, conn, T=1000, N=100, epsilon=0.2, rho=rho, alpha=alpha, chaos_measure='fft integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
plot_heatmap(S_test, alpha, rho, x_label_name='Alpha', y_label_name='Rho')

##### Grid Spike Count - rho alpha

In [ ]:
rho = [0.05, 0.25, 0.5, 0.75, 0.95, 1.1, 1.25, 1.5, 2, 2.5, 3]
alpha = [0, 0.00001, 0.01, 0.02, 0.05, 0.10, 0.15, 0.25, 0.5, 0.75, 0.95, 1, 1.1, 1.25, 1.5, 1.75, 2, 2.5]

In [ ]:
S_spikes = grid(F, conn, T=1000, N=100, epsilon=0.2, rho=rho, alpha=alpha, chaos_measure='spikes', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
# plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.3f' ):
plot_heatmap(S_spikes, alpha, rho,v_min=0, v_max=50,  x_label_name='Alpha', y_label_name='Rho', title="Number of Spikes of FFT above magnitude of 5", fmt='.0f')

##### Grid Integral of FFT - rho alpha

In [ ]:
rho = [0.05, 0.25, 0.5, 0.75, 0.95, 1.1, 1.25, 1.5, 2, 2.5, 3]
alpha = [0, 0.00001, 0.01, 0.02, 0.05, 0.10, 0.15, 0.25, 0.5, 0.75, 0.95, 1, 1.1, 1.25, 1.5, 1.75, 2]

In [ ]:
S_1 = grid(F, conn, T=1000, N=100, epsilon=0.2, rho=rho, alpha=alpha, chaos_measure='fft integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
# plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.3f' ):
plot_heatmap(S_1, alpha, rho, v_min=0, v_max= 0.2, x_label_name='Alpha', y_label_name='Rho', title="Integrals of FFT above magnitude of 5", fmt='.1f')

In [ ]:
S_1

##### Grid Integral of Pearson - rho alpha

In [ ]:
rho = [0.05, 0.25, 0.5, 0.8, 0.95, 1.1, 1.3, 1.5, 2, 2.5, 3]
alpha = [0, 0.00001, 0.01, 0.02, 0.05, 0.10, 0.15, 0.25, 0.5, 0.7, 0.8, 0.95, 1, 1.1, 1.5]

In [ ]:
S_1 = grid(F, conn, T=1000, N=100, epsilon=0.2, rho=rho, alpha=alpha, chaos_measure='pearson integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
# plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.3f' ):
plot_heatmap(S_1, alpha, rho, v_min=None, v_max= None, x_label_name='Alpha', y_label_name='Rho', title="Integrals of Pearson Correlation", fmt='.0f')

##### Grid Integral of FFT - N epsilon, rho=0.8

In [ ]:
N = [10, 50, 100, 150, 200]
epsilon = [0.1, 0.2, 0.4, 0.6, 0.8, 1]

In [ ]:
S_3 = grid(F, conn, T=1000, N=N, epsilon=epsilon, rho=0.8, alpha=0.8, chaos_measure='fft integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
# plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.3f' ):
plot_heatmap(S_3, epsilon, N, v_min=None, v_max= None, x_label_name='Sparsity', y_label_name='Number of Neurons', title="Integrals of FFT", fmt='.3f')

##### Grid Integral of FFT - N epsilon, rho=1.3

In [ ]:
N = [10, 50, 100, 150, 200]
epsilon = [0.1, 0.2, 0.4, 0.6, 0.8, 1]

In [ ]:
S_4 = grid(F, conn, T=1000, N=N, epsilon=epsilon, rho=1.3, alpha=0.8, chaos_measure='fft integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
# plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.3f' ):
plot_heatmap(S_4, epsilon, N, v_min=None, v_max= None, x_label_name='Sparsity', y_label_name='Number of Neurons', title="Integrals of FFT", fmt='.3f')

##### Grid Integral of Pearson - N epsilon, rho=0.8

In [ ]:
N = [10, 50, 100, 150, 200]
epsilon = [0.1, 0.2, 0.4, 0.6, 0.8, 1]

In [ ]:
S_5 = grid(F, conn, T=1000, N=N, epsilon=epsilon, rho=0.8, alpha=0.8, chaos_measure='pearson integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
# plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.3f' ):
plot_heatmap(S_5, epsilon, N, v_min=None, v_max= None, x_label_name='Sparsity', y_label_name='Number of Neurons', title="Integrals of FFT", fmt='.3f')

##### Grid Integral of Pearson - N epsilon, rho=0.8

In [ ]:
N = [10, 50, 100, 150, 200]
epsilon = [0.1, 0.2, 0.4, 0.6, 0.8, 1]

In [ ]:
S_6 = grid(F, conn, T=1000, N=N, epsilon=epsilon, rho=1.3, alpha=0.8, chaos_measure='pearson integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
# plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.3f' ):
plot_heatmap(S_6, epsilon, N, v_min=None, v_max= None, x_label_name='Sparsity', y_label_name='Number of Neurons', title="Integrals of FFT", fmt='.3f')

#### Electrical

In [ ]:
conn = "Electrical"

##### Integral of Pearson Correlation

In [ ]:
X, N, epsilon, rho, T, alpha, _ = compute_state_matrix(F, conn, 100, 0.2, 1.5, 1.1, 1000)
# Plots
states_average_plot(X, running_avg=True)
states_variance_plot(X)
fft_plot(X)
fft_plot(X, running_avg=True)
power_spectrum_plot(X)
pear_graph(X, future_time=1000)


In [ ]:
area_sum, integral = pearson_integral(X)
pear_graph(X, integrals=integral)

In [ ]:
print(area_sum, integral)

##### Integral of FFT

In [ ]:
# integral_of_spikes(X, treshold=10, window_size=10, running_avg= False, integral_analysis=False, f_min=None, f_max=None):
area_sum, integrals, spike_count = integral_of_spikes(X, threshold = 5, integral_analysis=True)
print(area_sum, spike_count)

In [ ]:
fft_plot(X, threshold=5,integrals=integrals)

##### Example of *Grid*

In [ ]:
rho = [0.05, 0.25, 0.5, 0.8, 0.95, 1.1, 1.3, 1.5, 2, 2.5, 3]
alpha = [0, 0.00001, 0.01, 0.2, 0.05, 0.10, 0.15, 0.25, 0.5, 0.7, 0.8, 0.95, 1, 1.1, 1.5,2]

In [ ]:
S_test = grid(F, conn, T=1000, N=100, epsilon=0.2, rho=rho, alpha=alpha, chaos_measure='fft integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
plot_heatmap(S_test, alpha, rho, x_label_name='Alpha', y_label_name='Rho')

##### Grid Integral of FFT - rho alpha

In [ ]:
rho = [0.05, 0.25, 0.5, 0.8, 0.95, 1.1, 1.3, 1.5, 2, 2.5, 3]
alpha = [0, 0.00001, 0.01, 0.02, 0.05, 0.10, 0.15, 0.25, 0.5, 0.7, 0.8, 0.95, 1, 1.1, 1.5]

In [ ]:
S_1 = grid(F, conn, T=1000, N=100, epsilon=0.2, rho=rho, alpha=alpha, chaos_measure='fft integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
# plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.3f' ):
plot_heatmap(S_1, alpha, rho, v_min=0, v_max= 0.25, x_label_name='Alpha', y_label_name='Rho', title="Integrals of FFT above magnitude of 5", fmt='.2f')

##### Grid Integral of Pearson - rho alpha

In [ ]:
rho = [0.05, 0.25, 0.5, 0.8, 0.95, 1.1, 1.3, 1.5, 2, 2.5, 3]
alpha = [0, 0.00001, 0.01, 0.02, 0.05, 0.10, 0.15, 0.25, 0.5, 0.7, 0.8, 0.95, 1, 1.1, 1.5]

In [ ]:
S_1 = grid(F, conn, T=1000, N=100, epsilon=0.2, rho=rho, alpha=alpha, chaos_measure='pearson integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
# plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.3f' ):
plot_heatmap(S_1, alpha, rho, v_min=None, v_max= None, x_label_name='Alpha', y_label_name='Rho', title="Integrals of Pearson Correlation", fmt='.0f')

##### Grid Integral of FFT - N epsilon, rho=0.8

In [ ]:
N = [10, 50, 100, 150, 200]
epsilon = [0.1, 0.2, 0.4, 0.6, 0.8, 1]

In [ ]:
S_3 = grid(F, conn, T=1000, N=N, epsilon=epsilon, rho=0.8, alpha=0.8, chaos_measure='fft integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
# plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.3f' ):
plot_heatmap(S_3, epsilon, N, v_min=None, v_max= None, x_label_name='Sparsity', y_label_name='Number of Neurons', title="Integrals of FFT", fmt='.3f')

##### Grid Integral of FFT - N epsilon, rho=1.3

In [ ]:
N = [10, 50, 100, 150, 200]
epsilon = [0.1, 0.2, 0.4, 0.6, 0.8, 1]

In [ ]:
S_4 = grid(F, conn, T=1000, N=N, epsilon=epsilon, rho=1.3, alpha=0.8, chaos_measure='fft integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
# plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.3f' ):
plot_heatmap(S_4, epsilon, N, v_min=None, v_max= None, x_label_name='Sparsity', y_label_name='Number of Neurons', title="Integrals of FFT", fmt='.3f')

##### Grid Integral of Pearson - N epsilon, rho=0.8

In [ ]:
N = [10, 50, 100, 150, 200]
epsilon = [0.1, 0.2, 0.4, 0.6, 0.8, 1]

In [ ]:
S_5 = grid(F, conn, T=1000, N=N, epsilon=epsilon, rho=0.8, alpha=0.8, chaos_measure='pearson integrals', treshold=5)

In [ ]:
# The later parameter given to grid() is rows
# plot_heatmap(data_matrix, x_labels, y_labels, v_min=None, v_max=None, x_label_name='X-axis', y_label_name='Y-axis', title='Heatmap', figsize=(10, 6), fmt='.3f' ):
plot_heatmap(S_5, epsilon, N, v_min=None, v_max= None, x_label_name='Sparsity', y_label_name='Number of Neurons', title="Integrals of FFT", fmt='.3f')

## Integrations

In [ ]:
def dynamical_syst_graph(*args):
    import matplotlib.pyplot as plt
    num_dimensions = len(args)

    # Create a figure
    fig = plt.figure(figsize=(10, 8))

    if num_dimensions == 1:
        # 1D plot
        plt.plot(args[0], lw=0.5, color='blue')
        plt.xlabel("Time Steps", color='black')
        plt.ylabel("X", color='black')
        plt.title("1D Dynamical System", color='black')

    elif num_dimensions == 2:
        # 2D plot
        plt.plot(args[0], args[1], lw=0.5, color='black')
        plt.xlabel("X", color='black', fontsize=16)
        plt.ylabel("Y", color='black', fontsize=16)
        #plt.title("", color='black')
        plt.tick_params(axis='both', labelsize=16)

    elif num_dimensions == 3:
        # 3D plot
        ax = fig.add_subplot(111, projection='3d', frame_on=False)
        ax.plot(args[0], args[1], args[2], lw=0.5, color='black')

        # Add a dot for the initial conditions
        ax.scatter([args[0][0]], [args[1][0]], [args[2][0]], color='red', s=25, label='Initial Condition')
        ax.legend()

        plt.tick_params(labelsize=12)
        ax.set_xlabel("X", color='black', fontsize=16)
        ax.set_ylabel("Y", color='black', fontsize=16)
        ax.set_zlabel("Z", color='black', fontsize=12)
        #ax.set_title("3D Dynamical System", color='black')

    else:
        raise ValueError("Unsupported number of dimensions. Please provide 1, 2, or 3 dimensions.")

    plt.show()

In [ ]:
trajectory = euler_method(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.005, 10000)
xs, ys, zs = zip(*trajectory)
dynamical_syst_graph(xs[1000:], ys[1000:])

In [ ]:
trajectory = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.005, 10000)
xs, ys, zs = zip(*trajectory)
dynamical_syst_graph(xs[1000:], ys[1000:])

### Interactive Graph


In [ ]:
dynamical_syst_interactive_graph(rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000), "RK4", 'Lorenz System')

### Sampling

In [ ]:
trajectory = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000)
xs, ys, zs = zip(*trajectory)
dynamical_syst_graph(xs, ys, zs)

In [ ]:
trajectory = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000, sampling_rate=10)
xs, ys, zs = zip(*trajectory)
dynamical_syst_graph(xs, ys, zs)

## Predict tau timestep in future rho=0.8 alpha=0.8 N=100, F=F_identity, input xyz


In [ ]:
r=0.8; a=0.8; N=100; F=F_identity; conn="Chemical"

### input x,y output x,y tau=1

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys, zs])
train_data = np.array([xs, ys, zs])

# def network(F, connectivity, input_data, output_data,
#            N, epsilon, rho, alpha,
#            T_transient, T_training, T_prediction, tau,
#            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=1,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

In [ ]:
type(lorenz_data)

In [ ]:
np.array(lorenz_data).shape

#### x prediction

In [ ]:
# states_average_plot_w_input_data_2(X, input_data, N, T, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200), tick_labelsize=18, axis_labelsize=18):

# states_average_plot_w_input_data(X, N, T,input_data, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200),tick_labelsize=18, axis_labelsize=18
states_average_plot_w_input_data(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Predicted Lorenz x",zoom_in=True, zoom_range=(750,1000))

In [ ]:
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Predicted Lorenz y",zoom_in=True, zoom_range=(750,1000))

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Predicted Lorenz z",zoom_in=True, zoom_range=(750,1000))

#### XYZ prediction


In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### input x,y output x,y tau=5

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys, zs])
train_data = np.array([xs, ys, zs])

# def network(F, connectivity, input_data, output_data,
#            N, epsilon, rho, alpha,
#            T_transient, T_training, T_prediction, tau,
#            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=5,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# states_average_plot_w_input_data_2(X, input_data, N, T, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200), tick_labelsize=18, axis_labelsize=18):

# states_average_plot_w_input_data(X, N, T,input_data, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200),tick_labelsize=18, axis_labelsize=18
states_average_plot_w_input_data(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Predicted Lorenz x",zoom_in=True, zoom_range=(750,1000))

In [ ]:
states_average_plot_w_input_data_3(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Predicted Lorenz x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Predicted Lorenz y",zoom_in=True, zoom_range=(750,1000))

In [ ]:
# Mean Field
states_average_plot_w_input_data_3(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Predicted Lorenz y",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Predicted Lorenz z",zoom_in=True, zoom_range=(750,1000))

### Input x,y Output x,y tau = 50

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 12000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys, zs])
train_data = np.array([xs, ys, zs])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=50,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo y", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### XYZ Prediction

In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### Input x,y Output x,y tau = 100

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 15000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys, zs])
train_data = np.array([xs, ys, zs])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=100,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### XYZ Prediction

In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### Input x,y Output x,y tau = 100 T_trainning=10000

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 15000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=10000, T_prediction=2000, tau=100,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo x", output_name="Prediction Lorenzo x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo y", output_name="Prediction Lorenzo y",zoom_in=True, zoom_range=(750,1000))

#### z prediction

### RMSE of Prediction wrt tau

In [ ]:
# Initialize lists to store tau values and corresponding RMSEs
taus = np.arange(1, 155, 5)  # Tau values from 0 to 150, step 5
rmses = []
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys, zs])
train_data = np.array([xs, ys, zs])
# Calculate RMSE for each tau
for tau in taus:
    real_values = xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
    _,_,_,_,_,_,_,_,predicted_values,_,_ = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=tau,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)


    overall_rmse, _ = rmse(real_values, predicted_values[0,:])
    rmses.append(overall_rmse)
    # Plotting
plt.figure(figsize=(10, 6))
plt.plot(taus, rmses, marker='o', color='black')
plt.xlabel('Tau', fontsize=14)
plt.ylabel('Overall RMSE', fontsize=14)
plt.tick_params(axis='both', labelsize=14)
plt.show()

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(taus, rmses, marker='o', color='black')
plt.xlabel('Tau', fontsize=14)
plt.ylabel('Overall RMSE', fontsize=14)
plt.tick_params(axis='both', labelsize=14)
plt.show()

## Predict tau timestep in future rho=0.8 alpha=0.8 N=100, F=F_identity, input xy


In [ ]:
r=0.8; a=0.8; N=100; F=F_identity; conn="Chemical"

### input x,y output x,y tau=1

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])

# def network(F, connectivity, input_data, output_data,
#            N, epsilon, rho, alpha,
#            T_transient, T_training, T_prediction, tau,
#            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=1,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# states_average_plot_w_input_data_2(X, input_data, N, T, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200), tick_labelsize=18, axis_labelsize=18):

# states_average_plot_w_input_data(X, N, T,input_data, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200),tick_labelsize=18, axis_labelsize=18
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Predicted Lorenz x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Predicted Lorenz z",zoom_in=True, zoom_range=(750,1000))

#### XYZ prediction


In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### input x,y output x,y tau=5

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys, zs])
train_data = np.array([xs, ys, zs])

# def network(F, connectivity, input_data, output_data,
#            N, epsilon, rho, alpha,
#            T_transient, T_training, T_prediction, tau,
#            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=5,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# states_average_plot_w_input_data_2(X, input_data, N, T, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200), tick_labelsize=18, axis_labelsize=18):

# states_average_plot_w_input_data(X, N, T,input_data, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200),tick_labelsize=18, axis_labelsize=18
states_average_plot_w_input_data(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Predicted Lorenz x",zoom_in=True, zoom_range=(750,1000))

In [ ]:
states_average_plot_w_input_data_3(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Predicted Lorenz x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Predicted Lorenz y",zoom_in=True, zoom_range=(750,1000))

In [ ]:
# Mean Field
states_average_plot_w_input_data_3(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Predicted Lorenz y",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Predicted Lorenz z",zoom_in=True, zoom_range=(750,1000))

### Input x,y Output x,y tau = 50

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 12000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=50,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo y", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### XYZ Prediction

In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### Input x,y Output x,y tau = 100

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 15000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=100,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### XYZ Prediction

In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### Input x,y Output x,y tau = 100 T_trainning=10000

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 15000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=10000, T_prediction=2000, tau=100,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo x", output_name="Prediction Lorenzo x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo y", output_name="Prediction Lorenzo y",zoom_in=True, zoom_range=(750,1000))

#### z prediction

### RMSE of Prediction wrt tau

In [ ]:
# Initialize lists to store tau values and corresponding RMSEs
taus = np.arange(1, 155, 15)  # Tau values from 0 to 150, step 5
rmses = []
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])
# Calculate RMSE for each tau
for tau in taus:
    real_values = xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
    _,_,_,_,_,_,_,_,predicted_values,_,_ = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=tau,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)


    overall_rmse, _ = rmse(real_values, predicted_values[0,:])
    rmses.append(overall_rmse)
    # Plotting
plt.figure(figsize=(10, 6))
plt.plot(taus, rmses, marker='o', color='black')
plt.xlabel('Tau', fontsize=14)
plt.ylabel('Overall RMSE', fontsize=14)
plt.tick_params(axis='both', labelsize=14)
plt.show()

## Predict tau timestep in future rho=0.8 alpha=np.linspace(0,0.25,15)[11] N=100, F=F_mod, conn="Electrical", input xy


In [ ]:
alphas = np.linspace(0,0.25,15)

In [ ]:
r=0.8; a=alphas[11]; N=100; F=F_mod; conn="Electrical"

### input x,y output x,y tau=1

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])

# def network(F, connectivity, input_data, output_data,
#            N, epsilon, rho, alpha,
#            T_transient, T_training, T_prediction, tau,
#            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=1,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# states_average_plot_w_input_data_2(X, input_data, N, T, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200), tick_labelsize=18, axis_labelsize=18):

# states_average_plot_w_input_data(X, N, T,input_data, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200),tick_labelsize=18, axis_labelsize=18
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Predicted Lorenz x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Predicted Lorenz z",zoom_in=True, zoom_range=(750,1000))

#### XYZ prediction


In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### input x,y output x,y tau=5

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys, zs])
train_data = np.array([xs, ys, zs])

# def network(F, connectivity, input_data, output_data,
#            N, epsilon, rho, alpha,
#            T_transient, T_training, T_prediction, tau,
#            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=5,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# states_average_plot_w_input_data_2(X, input_data, N, T, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200), tick_labelsize=18, axis_labelsize=18):

# states_average_plot_w_input_data(X, N, T,input_data, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200),tick_labelsize=18, axis_labelsize=18
states_average_plot_w_input_data(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Predicted Lorenz x",zoom_in=True, zoom_range=(750,1000))

In [ ]:
states_average_plot_w_input_data_3(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Predicted Lorenz x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Predicted Lorenz y",zoom_in=True, zoom_range=(750,1000))

In [ ]:
# Mean Field
states_average_plot_w_input_data_3(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Predicted Lorenz y",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Predicted Lorenz z",zoom_in=True, zoom_range=(750,1000))

### Input x,y Output x,y tau = 50

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 12000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=50,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo y", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### XYZ Prediction

In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### Input x,y Output x,y tau = 100

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 15000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=100,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### XYZ Prediction

In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### Input x,y Output x,y tau = 100 T_trainning=10000

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 15000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=10000, T_prediction=2000, tau=100,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo x", output_name="Prediction Lorenzo x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo y", output_name="Prediction Lorenzo y",zoom_in=True, zoom_range=(750,1000))

#### z prediction

### RMSE of Prediction wrt tau

In [ ]:
# Initialize lists to store tau values and corresponding RMSEs
taus = np.arange(1, 155, 15)  # Tau values from 0 to 150, step 5
rmses = []
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])
# Calculate RMSE for each tau
for tau in taus:
    real_values = xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
    _,_,_,_,_,_,_,_,predicted_values,_,_ = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=tau,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)


    overall_rmse, _ = rmse(real_values, predicted_values[0,:])
    rmses.append(overall_rmse)
    # Plotting
plt.figure(figsize=(10, 6))
plt.plot(taus, rmses, marker='o', color='black')
plt.xlabel('Tau', fontsize=14)
plt.ylabel('Overall RMSE', fontsize=14)
plt.tick_params(axis='both', labelsize=14)
plt.show()

## Predict tau timestep in future rho=0.8 alpha=0.8 N=100, F=F_identity, input xy with sampling

In [ ]:
r=0.8; a=0.8; N=100; F=F_identity; conn="Chemical"

### input x,y output x,y tau=1

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 50000, sampling_rate=5)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])

# def network(F, connectivity, input_data, output_data,
#            N, epsilon, rho, alpha,
#            T_transient, T_training, T_prediction, tau,
#            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=1,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

In [ ]:
dynamical_syst_graph(xs[:2000], ys[:2000], zs[:2000])

#### x prediction

In [ ]:
# states_average_plot_w_input_data_2(X, input_data, N, T, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200), tick_labelsize=18, axis_labelsize=18):

# states_average_plot_w_input_data(X, N, T,input_data, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200),tick_labelsize=18, axis_labelsize=18
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Predicted Lorenz x",zoom_in=True, zoom_range=(750,850))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Prediction",zoom_in=True, zoom_range=(750,850))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Predicted Lorenz z",zoom_in=True, zoom_range=(750,850))

#### XYZ prediction


In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### input x,y output x,y tau=5

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys, zs])
train_data = np.array([xs, ys, zs])

# def network(F, connectivity, input_data, output_data,
#            N, epsilon, rho, alpha,
#            T_transient, T_training, T_prediction, tau,
#            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=5,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# states_average_plot_w_input_data_2(X, input_data, N, T, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200), tick_labelsize=18, axis_labelsize=18):

# states_average_plot_w_input_data(X, N, T,input_data, input_name, output_name="Average States", title=None, t_transient=0,
# t_end=None, window_size=10, running_avg=False, zoom_in=False, zoom_range=(100, 200),tick_labelsize=18, axis_labelsize=18
states_average_plot_w_input_data(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Predicted Lorenz x",zoom_in=True, zoom_range=(750,1000))

In [ ]:
states_average_plot_w_input_data_3(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Predicted Lorenz x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Predicted Lorenz y",zoom_in=True, zoom_range=(750,1000))

In [ ]:
# Mean Field
states_average_plot_w_input_data_3(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Predicted Lorenz y",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Predicted Lorenz z",zoom_in=True, zoom_range=(750,1000))

### Input x,y Output x,y tau = 50

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 50000, sampling_rate=5)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=50,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction",zoom_in=True, zoom_range=(750,850))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo y", output_name="Prediction",zoom_in=True, zoom_range=(750,850))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Prediction",zoom_in=True, zoom_range=(750,850))

#### XYZ Prediction

In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### Input x,y Output x,y tau = 100

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 15000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=100,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### XYZ Prediction

In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### Input x,y Output x,y tau = 100 T_trainning=10000

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 15000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=10000, T_prediction=2000, tau=100,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo x", output_name="Prediction Lorenzo x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo y", output_name="Prediction Lorenzo y",zoom_in=True, zoom_range=(750,1000))

#### z prediction

### RMSE of Prediction wrt tau

In [ ]:
# Initialize lists to store tau values and corresponding RMSEs
taus = np.arange(1, 155, 15)  # Tau values from 0 to 150, step 5
rmses_samp = []
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 50000, sampling_rate=5)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])
# Calculate RMSE for each tau
for tau in taus:
    real_values = xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
    _,_,_,_,_,_,_,_,predicted_values,_,_ = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=tau,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)


    overall_rmse, _ = rmse(real_values, predicted_values[0,:])
    rmses_samp.append(overall_rmse)
    # Plotting
plt.figure(figsize=(10, 6))
plt.plot(taus, rmses_samp, marker='o', color='black')
plt.xlabel('Tau', fontsize=14)
plt.ylabel('Overall RMSE', fontsize=14)
plt.tick_params(axis='both', labelsize=14)
plt.show()

## Predict tau timestep in future rho=0.8 alpha=0.01 N=100, F=F_mod

In [ ]:
r=0.8; a=0.8; N=100

### input x,y output x,y tau=5

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 4000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
    N=N, epsilon=0.2, rho=r, alpha=a, T_transient=500, T_training=2000, T_prediction=1000, tau=5,
    input_data=input_data, output_data=train_data, b_in=1, b_out=1, inside_states=False, F=F_mod, regulization=0, ex_time=False)


#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction Lorenzo x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Prediction Lorenzo y",zoom_in=True, zoom_range=(750,1000))

#### z prediction

### Input x,y Output x,y tau = 50

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 2000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys, zs])

# Reservoir
#def network(N, epsilon, rho, alpha,T_transient, T_training, T_prediction
                                         #,tau, input_data, output_data, b_in, b_out,inside_states=False, F=F_identity,
                                         #regulization=0, ex_time=False):
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
    N=N, epsilon=0.2, rho=r, alpha=a, T_transient=100, T_training=1000, T_prediction=500, tau=50,
    input_data=input_data, output_data=train_data, b_in=1, b_out=1, inside_states=False, F=F_mod, regulization=0, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo x", output_name="Prediction Lorenzo x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo y", output_name="Prediction Lorenzo y",zoom_in=True, zoom_range=(750,1000))

### Input x,y Output x,y tau = 100

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 12000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

# Reservoir
X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = network(
    N=N, epsilon=0.2, rho=r, alpha=a, T_transient=2000, T_training=5000, T_prediction=2000, tau=100,
    input_data=input_data, output_data=train_data, b_in=1, b_out=1, regulization=0, F=F_mod, ex_time=False)

#### x prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo x", output_name="Prediction Lorenzo x",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo y", output_name="Prediction Lorenzo y",zoom_in=True, zoom_range=(750,1000))

In [ ]:
# Mean Field
states_average_plot_w_input_data(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenzo y", output_name="Prediction Lorenzo y",zoom_in=True, zoom_range=(750,1000))

#### z prediction

### RMSE of Prediction wrt tau

In [ ]:
# Initialize lists to store tau values and corresponding RMSEs
taus = np.arange(0, 155, 5)  # Tau values from 0 to 150, step 5
rmses = []

# Calculate RMSE for each tau
for tau in taus:
    real_values = xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
    _,_,_,_,_,_,_,_,predicted_values,_,_ = network(
    N=N, epsilon=0.2, rho=r, alpha=a, T_transient=2000, T_training=5000, T_prediction=2000, tau=tau,
    input_data=input_data, output_data=train_data, F=F_mod, b_in=1, b_out=1, regulization=0, ex_time=False)


    overall_rmse, _ = rmse(real_values, predicted_values[0,:])
    rmses.append(overall_rmse)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(taus, rmses, marker='o')
plt.xlabel('Tau')
plt.ylabel('Overall RMSE')
plt.title('Overall RMSE vs Tau')
plt.grid(True)
plt.show()

## Grids

In [ ]:
def grid_prediction(F,conn, input_data, train_data,
                    N=100, epsilon=0.2, rho=0.8, alpha=0.8,
                    T_transient=2000, T_training=5000, T_prediction=2000, tau=1,
                    Lapl=True, regulization=0, b_in=1, b_out=1, color='RMSE'):

    # With this function you can grid whatever you need for prediction
    # Color is written like that as it is color the heatmap will use
    # Here if grid is N and rho then
    # S rows will be wrt to rho
    # S columns will be wrt to N

    S = []
    hyperparameters = {'N': N, 'epsilon': epsilon, 'alpha': alpha, 'rho': rho }
    par_to_grid = []

    for key, value in hyperparameters.items():
        if isinstance(value, list):
            par_to_grid.append((key, value))

    class CustomError(Exception):
        pass

    if len(par_to_grid) != 2:
        raise CustomError("This function accepts only 2 parameters as lists!")

    key1, var1 = par_to_grid[0]
    key2, var2 = par_to_grid[1]

    total_iterations = len(var1) * len(var2)
    current_iteration = 0

    for i in range(len(var1)):
        row = []
        for j in range(len(var2)):
            current_iteration += 1
            print(f"Processing {key1}={var1[i]}, {key2}={var2[j]} ... ({current_iteration}/{total_iterations})")

            # Update the hyperparameters dictionary
            hyperparameters[key1] = var1[i]
            hyperparameters[key2] = var2[j]


            X,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  **hyperparameters,
                                                  T_transient=T_transient, T_training=T_training, T_prediction=T_prediction, tau=tau,
                                                  Lapl=Lapl, regulization=regulization, b_in=b_in, b_out=b_out, ex_time=False)
            """
            if color == 'integrals':
              row.append(integral_of_spikes(X, T, treshold))
            elif color == 'spikes':
              row.append(spike_count(X,T,treshold))
            elif color == 'pearson':
              row.append(pear_in_ft(X, future_time=100))
            elif color == 'pearson integrals':
              row.append(pearson_integral(X, integral_analysis=False))
            """
            if color == 'RMSE':
              overall_rmse, _ = rmse(input_data[0,(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)], Y[0,:])
              row.append(overall_rmse)
            else:
              raise CustomError(f"{color} is invalid: please specify -integrals- or -spikes- or -pearson- or -RMSE-")
        S.append(row)

    return S


### Rho-alpha, tau=1

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 12000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

rho_list = [0.05, 0.25, 0.5, 0.75, 0.95, 1.1, 1.25, 1.5, 2, 2.5, 3]
alpha_list = [0, 0.00001, 0.01, 0.02, 0.05, 0.10, 0.15, 0.25, 0.5, 0.75, 0.95, 1, 1.1, 1.25, 1.5, 1.75, 2]

S_r_a_tau1 = grid_prediction(F,conn, input_data, train_data,
                    N=100, epsilon=0.2, rho=rho_list, alpha=alpha_list,
                    T_transient=2000, T_training=5000, T_prediction=2000, tau=1,
                    Lapl=True, regulization=0, b_in=1, b_out=1, color='RMSE')

In [ ]:
#Transpose matrix
S_r_a_tau1 = np.array(S_r_a_tau1)
S_r_a_tau1 = S_r_a_tau1.T

In [ ]:
plot_heatmap(S_r_a_tau1, rho_list,alpha_list, v_min=0, v_max= 0.1, x_label_name='Rho', y_label_name='Alpha', title="RMSE of Reservoirs", fmt='.2f')

### Rho-alpha, tau 25

In [ ]:
# Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 12000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

rho_list = [0.05, 0.25, 0.5, 0.75, 0.95, 1.1, 1.25, 1.5, 2, 2.5, 3]
alpha_list = [0, 0.00001, 0.01, 0.02, 0.05, 0.10, 0.15, 0.25, 0.5, 0.75, 0.95, 1, 1.1, 1.25, 1.5, 1.75, 2]

S_r_a_tau25 = grid_prediction(F,conn, input_data, train_data,
                    N=100, epsilon=0.2, rho=rho_list, alpha=alpha_list,
                    T_transient=2000, T_training=5000, T_prediction=2000, tau=25,
                    Lapl=True, regulization=0, b_in=1, b_out=1, color='RMSE')

In [ ]:
plot_heatmap(S_r_a_tau25, rho_list,alpha_list, v_min=0, v_max= 6, x_label_name='Rho', y_label_name='Alpha', title="RMSE of Reservoirs", fmt='.2f')

### Rho-Alpha, tau 50

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 12000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

rho_list = [0.05, 0.25, 0.5, 0.75, 0.95, 1.1, 1.25, 1.5, 2, 2.5, 3]
alpha_list = [0, 0.00001, 0.01, 0.02, 0.05, 0.10, 0.15, 0.25, 0.5, 0.75, 0.95, 1, 1.1, 1.25, 1.5, 1.75, 2]

S_r_a_tau50 = grid_prediction(F,conn, input_data, train_data,
                    N=100, epsilon=0.2, rho=rho_list, alpha=alpha_list,
                    T_transient=2000, T_training=5000, T_prediction=2000, tau=50,
                    Lapl=True, regulization=0, b_in=1, b_out=1, color='RMSE')

In [ ]:
plot_heatmap(S_r_a_tau50, rho_list,alpha_list, v_min=0, v_max= 6, x_label_name='Rho', y_label_name='Alpha', title="RMSE of Reservoirs", fmt='.2f')

### Rho-Training, tau1

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 12000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

rho_list = [0.05, 0.25, 0.5, 0.75, 0.95, 1.1, 1.25, 1.5, 2, 2.5, 3]
alpha_list = [0, 0.00001, 0.01, 0.02, 0.05, 0.10, 0.15, 0.25, 0.5, 0.75, 0.95, 1, 1.1, 1.25, 1.5, 1.75, 2, 2.5]

S_r_a_tau1 = grid_prediction(F,conn, input_data, train_data,
                    N=100, epsilon=0.2, rho=rho_list, alpha=alpha_list,
                    T_transient=2000, T_training=5000, T_prediction=2000, tau=1,
                    Lapl=True, regulization=0, b_in=1, b_out=1, color='RMSE')

In [ ]:
    S = []
    hyperparameters = {'N': N, 'epsilon': epsilon, 'alpha': alpha, 'rho': rho }
    par_to_grid = []

    for key, value in hyperparameters.items():
        if isinstance(value, list):
            par_to_grid.append((key, value))

    class CustomError(Exception):
        pass

    if len(par_to_grid) != 2:
        raise CustomError("This function accepts only 2 parameters as lists!")

    key1, var1 = par_to_grid[0]
    key2, var2 = par_to_grid[1]

    total_iterations = len(var1) * len(var2)
    current_iteration = 0

    for i in range(len(var1)):
        row = []
        for j in range(len(var2)):
            current_iteration += 1
            print(f"Processing {key1}={var1[i]}, {key2}={var2[j]} ... ({current_iteration}/{total_iterations})")

            # Update the hyperparameters dictionary
            hyperparameters[key1] = var1[i]
            hyperparameters[key2] = var2[j]


            X,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y, tau, W = network(
                                                  F, conn, input_data, train_data,
                                                  **hyperparameters,
                                                  T_transient=T_transient, T_training=T_training, T_prediction=T_prediction, tau=tau,
                                                  Lapl=Lapl, regulization=regulization, b_in=b_in, b_out=b_out, ex_time=False)
            """
            if color == 'integrals':
              row.append(integral_of_spikes(X, T, treshold))
            elif color == 'spikes':
              row.append(spike_count(X,T,treshold))
            elif color == 'pearson':
              row.append(pear_in_ft(X, future_time=100))
            elif color == 'pearson integrals':
              row.append(pearson_integral(X, integral_analysis=False))
            """
            if color == 'RMSE':
              overall_rmse, _ = rmse(input_data[0,(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)], Y[0,:])
              row.append(overall_rmse)
            else:
              raise CustomError(f"{color} is invalid: please specify -integrals- or -spikes- or -pearson- or -RMSE-")
        S.append(row)

    return S

### Grid tau=1

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 12000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

N_list = [50,100,150,200,250]
rho_list = [0.5,0.8,0.95,1.1,1.5,1.8,2.5]

S_pred_t1 = grid_prediction(input_data, train_data, T_transient=2000, T_training=5000, T_prediction=2000,
                    N=N_list, epsilon=0.2, rho=rho_list, alpha=0.8, tau=1,
                    b_in=1, b_out=1,regulization=0, color='RMSE', F=F_mod)

In [ ]:
# AS S_pred rows are wrt to rho we want rho to be in x axis
# AS S_pred columns are made wrt to N, lets put N into y axis
plot_heatmap(S_pred_t1, rho_list,N_list, x_label_name='Rho', y_label_name='Number of Neurons',
             title='Heatmap', figsize=(10, 6), fmt='.5f')

### Grid tau=25

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 12000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

N_list = [50,100,150,200,250]
rho_list = [0.5,0.8,0.95,1.1,1.5,1.8,2.5]

S_pred_t25 = grid_prediction(input_data, train_data, T_transient=2000, T_training=5000, T_prediction=2000,
                    N=N_list, epsilon=0.2, rho=rho_list, alpha=0.8, tau=25,
                    b_in=1, b_out=1,regulization=0, color='RMSE', F=F_mod)

In [ ]:
# AS S_pred rows are wrt to rho we want rho to be in x axis
# AS S_pred columns are made wrt to N, lets put N into y axis
plot_heatmap(S_pred_t25, rho_list,N_list, x_label_name='Rho', y_label_name='Number of Neurons',
             title='Heatmap', figsize=(10, 6), fmt='.5f')

### Grid tau=50

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 12000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

N_list = [50,100,150,200,250]
rho_list = [0.5,0.8,0.95,1.1,1.5,1.8,2.5]

S_pred_t50 = grid_prediction(input_data, train_data, T_transient=2000, T_training=5000, T_prediction=2000,
                    N=N_list, epsilon=0.2, rho=rho_list, alpha=0.8, tau=50,
                    b_in=1, b_out=1,regulization=0, color='RMSE', F=F_mod)

In [ ]:
# AS S_pred rows are wrt to rho we want rho to be in x axis
# AS S_pred columns are made wrt to N, lets put N into y axis
plot_heatmap(S_pred_t50, rho_list,N_list, x_label_name='Rho', y_label_name='Number of Neurons',
             title='Heatmap', figsize=(10, 6), fmt='.5f')

# Looped Network

## Predict tau timestep in future rho=0.8 alpha=0.8 N=100, F=F_identity, input xy, T_trainning=5000


In [ ]:
r=0.8; a=0.8; N=100; F=F_identity; conn="Chemical"

### input x,y output x,y tau=1

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 20000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

# def network(F, connectivity, input_data, output_data,
#            N, epsilon, rho, alpha,
#            T_transient, T_training, T_prediction, tau,
#            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = looped_network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=1,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction",zoom_in=True, zoom_range=(100,400))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Prediction",zoom_in=True, zoom_range=(100,400))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Predicted Lorenz z",zoom_in=True, zoom_range=(750,1000))

#### XYZ prediction


In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

### RMSE of Prediction wrt tau

In [ ]:
# Initialize lists to store tau values and corresponding RMSEs
taus = np.arange(1, 155, 5)  # Tau values from 0 to 150, step 5
rmses = []
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 10000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys, zs])
train_data = np.array([xs, ys, zs])
# Calculate RMSE for each tau
for tau in taus:
    real_values = xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
    _,_,_,_,_,_,_,_,predicted_values,_,_ = network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=5000, T_prediction=2000, tau=tau,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)


    overall_rmse, _ = rmse(real_values, predicted_values[0,:])
    rmses.append(overall_rmse)
    # Plotting
plt.figure(figsize=(10, 6))
plt.plot(taus, rmses, marker='o', color='black')
plt.xlabel('Tau', fontsize=14)
plt.ylabel('Overall RMSE', fontsize=14)
plt.tick_params(axis='both', labelsize=14)
plt.show()

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(taus, rmses, marker='o', color='black')
plt.xlabel('Tau', fontsize=14)
plt.ylabel('Overall RMSE', fontsize=14)
plt.tick_params(axis='both', labelsize=14)
plt.show()

## Predict tau timestep in future rho=0.8 alpha=0.8 N=100, F=F_identity, input xy, T_trainning=15000


In [ ]:
r=0.8; a=0.8; N=100; F=F_identity; conn="Chemical"

### input x,y output x,y tau=1

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 20000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

# def network(F, connectivity, input_data, output_data,
#            N, epsilon, rho, alpha,
#            T_transient, T_training, T_prediction, tau,
#            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = looped_network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=15000, T_prediction=2000, tau=1,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Prediction",zoom_in=True, zoom_range=(750,1000))

#### z prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[2,:].reshape(1,-1),N,T_prediction,zs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz z", output_name="Predicted Lorenz z",zoom_in=True, zoom_range=(750,1000))

#### XYZ prediction


In [ ]:
dynamical_syst_graph(Y_pred[0,:], Y_pred[1,:], Y_pred[2,:])

In [ ]:
x = np.squeeze(Y_pred[0,:])
y = np.squeeze(Y_pred[1,:])
z = np.squeeze(Y_pred[2,:])
dynamical_syst_interactive_graph(x, y, z, "Network Prediction", "Lorenz system")

## Predict tau timestep in future rho=0.8 alpha=0.8 N=100, F=F_identity, input xy, T_trainning=45000


In [ ]:
r=0.8; a=0.8; N=100; F=F_identity; conn="Chemical"

### input x,y output x,y tau=1

In [ ]:
 # Data
lorenz_data = rk4(lorenz_system, 0, 1, 1, 1, 10, 8/3, 28, 0.01, 50000)
xs, ys, zs = zip(*lorenz_data)
input_data = np.array([xs, ys])
train_data = np.array([xs, ys])

# def network(F, connectivity, input_data, output_data,
#            N, epsilon, rho, alpha,
#            T_transient, T_training, T_prediction, tau,
#            Lapl= True, regulization=0, b_in=1, b_out=1, ex_time=False):

X_pred,N,epsilon,rho,T_transient, T_training, T_prediction, alpha, Y_pred, tau, W = looped_network(
                                                  F, conn, input_data, train_data,
                                                  N=N, epsilon=0.2, rho=r, alpha=a,
                                                  T_transient=2000, T_training=45000, T_prediction=2000, tau=1,
                                                  Lapl=True, regulization=0, b_in=1, b_out=1, ex_time=False)

#### x prediction

In [ ]:
states_average_plot_w_input_data_2(Y_pred[0,:].reshape(1,-1),N,T_prediction,xs[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz x", output_name="Prediction",zoom_in=True, zoom_range=(0,250))

#### y prediction

In [ ]:
# Mean Field
states_average_plot_w_input_data_2(Y_pred[1,:].reshape(1,-1),N,T_prediction,ys[(T_transient+T_training+tau):(T_transient+T_training+T_prediction+tau)]
                                 ,input_name="Lorenz y", output_name="Prediction",zoom_in=True, zoom_range=(0,250))

In [ ]:
pear_graph(Y_pred[1,:].reshape(1,-1), future_time=1000)

# Bifurations

## F=F_identity

In [ ]:
F = F_identity

## Chemical

In [ ]:
conn = "Chemical"

### Network 1, N=100, sparsity=0.2, rho=0.8. alpha[0:2]

In [ ]:
B_1 = bifurcations_wrt_alpha(F, conn, 100, 0.2, 0.8, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_1
plot_bifurcation(param_list, spikes)

In [ ]:
B_1_1 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [5,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

#### alpha[0-1.75]

In [ ]:
B_1_2 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0, 1.75, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [5,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

#### alpha[0-1.9]

In [ ]:
B_1_2 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0, 1.9, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [5,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

#### alpha[0, 0.25]

In [ ]:
B_1_2 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0, 0.25, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [5,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

### Network 2, N=100, e=0.2, rho=[0.0001, 3], alpha=0.8

In [ ]:
B_2 = bifurcations_wrt_rho(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 0.8,                           # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_2
plot_bifurcation(param_list, spikes)

In [ ]:
B_2_1 = bifurcations_wrt_rho_2(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 0.8,                           # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, [5,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
run_network_and_plot_bifurcation_2(compute_state_matrix, F_mod, vec_of_spec_X_row_weighted, find_spikes, np.linspace(0, 0.15, 20), [5, 95])

### Network 3, N=100, e=0.2, rho=[0.000001, 3], alpha=1.3

In [ ]:
B_3 = bifurcations_wrt_rho(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 1.3,                       # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_3
plot_bifurcation(param_list, spikes)

In [ ]:
B_3_1 = bifurcations_wrt_rho_2(F, conn, 100, 0.2,np.linspace(0.00001, 3, 30), 1.3,                           # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, [5,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)

### Network 4, N=100, e=0.2, rho=[0.000001, 3], alpha=1.8

In [ ]:
B_4 = bifurcations_wrt_rho(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 1.8,                       # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_4
plot_bifurcation(param_list, spikes)

### Network 5, N=100, sparsity=0.2, rho=1.3, alpha[0:2]

In [ ]:
B_5 = bifurcations_wrt_alpha(F, conn, 100, 0.2, 1.3, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_5
plot_bifurcation(param_list, spikes)

### Network 6, N=100, sparsity=0.2, rho=1.3, alpha[0:2]

In [ ]:
B_6 = bifurcations_wrt_alpha(F, conn, 100, 0.2, 2.5, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_6
plot_bifurcation(param_list, spikes)

## F=F_identity, conn="Electrical"

In [ ]:
F=F_identity

In [ ]:
conn = "Electrical"

### Network 1, N=100, sparsity=0.2, rho=0.8. alpha[0:2]

In [ ]:
B_1 = bifurcations_wrt_alpha(F, conn, 100, 0.2, 0.8, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_1
plot_bifurcation(param_list, spikes)

In [ ]:
B_1_1 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0, 1.95, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

#### alpha[1.9 -1.98]

In [ ]:
B_1_1 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(1.9, 1.98, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

### Network 2, N=100, e=0.2, rho=[0.0001, 3], alpha=0.8

In [ ]:
B_2 = bifurcations_wrt_rho(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 0.8,                           # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_2
plot_bifurcation(param_list, spikes)

In [ ]:
B_2_1 = bifurcations_wrt_rho_2(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 0.8,                           # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, [5,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
run_network_and_plot_bifurcation_2(compute_state_matrix, F_mod, vec_of_spec_X_row_weighted, find_spikes, np.linspace(0, 0.15, 20), [5, 95])

### Network 3, N=100, e=0.2, rho=[0.000001, 3], alpha=1.3

In [ ]:
B_3 = bifurcations_wrt_rho(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 1.3,                       # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_3
plot_bifurcation(param_list, spikes)

### Network 4, N=100, e=0.2, rho=[0.000001, 3], alpha=1.8

In [ ]:
B_4 = bifurcations_wrt_rho(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 1.8,                       # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_4
plot_bifurcation(param_list, spikes)

### Network 5, N=100, sparsity=0.2, rho=1.3, alpha[0:2]

In [ ]:
B_5 = bifurcations_wrt_alpha(F, conn, 100, 0.2, 1.3, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_5
plot_bifurcation(param_list, spikes)

### Network 6, N=100, sparsity=0.2, rho=2.5, alpha[0:2]

In [ ]:
B_6 = bifurcations_wrt_alpha(F, conn, 100, 0.2, 2.5, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_6
plot_bifurcation(param_list, spikes)

## F=F_mod, conn="Chemical"

In [ ]:
F=F_mod

In [ ]:
conn = "Chemical"

### Network 1, N=100, sparsity=0.2, rho=0.8. alpha[0:2]

In [ ]:
B_1 = bifurcations_wrt_alpha(F, conn, 100, 0.2, 0.8, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_1
plot_bifurcation(param_list, spikes)

In [ ]:
B_1_1 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [5,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

#### alpha[0-0.25]


In [ ]:
B_1_2 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0, 0.25, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [5,50,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
B_1_2 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0, 0.25, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [85,97],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

#### alpha[0.05-0.08]

In [ ]:
B_1_2 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0.05, 0.08, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [85,97],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

#### alpha[0.18 - 0.23]

In [ ]:
B_1_2 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0.18, 0.23, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [85,97],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
B_1_2 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0.18, 0.23, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [1,50,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

### Network 2, N=100, e=0.2, rho=[0.0001, 3], alpha=0.8

In [ ]:
B_2 = bifurcations_wrt_rho(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 0.8,                           # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_2
plot_bifurcation(param_list, spikes)

In [ ]:
B_2_1 = bifurcations_wrt_rho_2(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 0.8,                           # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, [5,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

### Network 3, N=100, e=0.2, rho=[0.000001, 3], alpha=1.3

In [ ]:
B_3 = bifurcations_wrt_rho(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 1.3,                       # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_3
plot_bifurcation(param_list, spikes)

### Network 4, N=100, e=0.2, rho=[0.000001, 3], alpha=1.8

In [ ]:
B_4 = bifurcations_wrt_rho(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 1.8,                       # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_4
plot_bifurcation(param_list, spikes)

### Network 5, N=100, sparsity=0.2, rho=1.3, alpha[0:2]

In [ ]:
B_5 = bifurcations_wrt_alpha(F, conn, 100, 0.2, 1.3, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_5
plot_bifurcation(param_list, spikes)

### Network 6, N=100, sparsity=0.2, rho=2.5, alpha[0:2]

In [ ]:
B_6 = bifurcations_wrt_alpha(F, conn, 100, 0.2, 2.5, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_6
plot_bifurcation(param_list, spikes)

## F=F_mod, conn="Electrical"

In [ ]:
F=F_mod

In [ ]:
conn = "Electrical"

### Network 1, N=100, sparsity=0.2, rho=0.8. alpha[0:2]

In [ ]:
B_1 = bifurcations_wrt_alpha(F, conn, 100, 0.2, 0.8, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_1
plot_bifurcation(param_list, spikes)

In [ ]:
B_1_1 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [5,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

#### alpha[0,0.25]

In [ ]:
B_1_2 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0, 0.25, 15),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [5,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
B_1_3 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0, 0.25, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [10,90],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

#### alpha[0, 1.75] 2 neurons

In [ ]:
B_1_4 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0, 1.75, 60),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [10,90],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

### Network 2, N=100, e=0.2, rho=[0.0001, 3], alpha=0.8

In [ ]:
B_2 = bifurcations_wrt_rho(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 0.8,                           # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_2
plot_bifurcation(param_list, spikes)

In [ ]:
B_2_1 = bifurcations_wrt_rho_2(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 0.8,                           # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, [5,95],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

### Network 3, N=100, e=0.2, rho=[0.000001, 3], alpha=1.3

In [ ]:
B_3 = bifurcations_wrt_rho(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 1.3,                       # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_3
plot_bifurcation(param_list, spikes)

### Network 4, N=100, e=0.2, rho=[0.000001, 3], alpha=1.8

In [ ]:
B_4 = bifurcations_wrt_rho(F, conn, 100, 0.2, np.linspace(0.00001, 3, 30), 1.8,                       # F=F_identity, connectivity, Neurons, sparsity, rhos, alpha
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_4
plot_bifurcation(param_list, spikes)

### Network 5, N=100, sparsity=0.2, rho=1.3, alpha[0:2]

In [ ]:
B_5 = bifurcations_wrt_alpha(F, conn, 100, 0.2, 1.3, np.linspace(0, 2, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, 95,                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_5
plot_bifurcation(param_list, spikes)

In [ ]:
B_1_5 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 1.3, np.linspace(0, 0.25, 30),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [10,90],                                                  # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

### Network 6, N=100, sparsity=0.2, np.linspace(0.00001, 3,60), np.linspace(0, 1.75, 60)[9]

In [ ]:
alphas = np.linspace(0, 1.75, 60)

In [ ]:
alphas[9]

In [ ]:
B_6 = bifurcations_wrt_rho_2(F, conn, 100, 0.2, np.linspace(0.00001, 3,60), alphas[9],                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [10,90],                                               # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

In [ ]:
param_list, spikes = B_6
plot_bifurcation(param_list, spikes)

#### Network 7 alpha[0-0.05], rho=0.8, ind[10,85,97]

In [ ]:
B_7 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0.00000001, 0.05,60),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [10, 85, 97],                                               # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

#### Network 8 alpha[0-0.01], rho=0.8, ind[10,85,97]

In [ ]:
B_8 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0.00000001, 0.01,60),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [10, 85, 97],                                               # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

#### Network 8 alpha[0-0.01], rho=0.8, ind[10,38,46]

In [ ]:
B_9 = bifurcations_wrt_alpha_2(F, conn, 100, 0.2, 0.8, np.linspace(0.00000001, 0.01,60),                               # F=F_identity, connectivity, Neurons, sparsity, rho, alphas
                                     1000, 2000, [10, 38, 46],                                               # T_transient, T = all time, Neuron index,
                                      compute_state_matrix, vec_of_spec_X_row_weighted, find_spikes)  # functions

# Individual neurons investigation

## F=F_identity, conn="Chemical"

In [ ]:
F=F_identity
conn = "Chemical"

### Network 1 rho=0.8, alpha=0.8

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 0.8, 0.8, 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
degrees_distribution(W)
weighted_degrees_distribution(W, N)
plot_weighted_degrees(weighted_degrees(W,N), "Sorted Neuron Index")
plot_weighted_degrees(weighted_degrees_unsorted(W,N), "Neuron Index")

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 70, 0.01)
plot_X_similar_vec(X, similar_indices)

### Network 2 rho=np.linspace(0.00001, 3, 30)[10], alpha=0.8

In [ ]:
np.linspace(0.00001, 3, 30)[15]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2,np.linspace(0.00001, 3, 30)[15], 0.8, 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
plot_weighted_degrees(weighted_degrees(W,N))

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 70, 0.01)
plot_X_similar_vec(X, [5,95],legend=True, W=W, N=N)

In [ ]:
plot_X_similar_vec(X, [5,95, 1,2,3,4,5,6],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

In [ ]:
plot_X_similar_vec(X, [1,95,4],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.08, 0.01)
plot_X_similar_vec(X, similar_indices,start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

## F=F_mod, conn = "Chemical"

In [ ]:
F=F_mod
conn = "Chemical"

### Network 1 rho=0.8, alpha = np.linspace(0, 2, 30)[17]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2,0.8, np.linspace(0, 2, 30)[17], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
plot_weighted_degrees(weighted_degrees(W,N))

In [ ]:
plot_X_similar_vec(X, [5,95],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)

In [ ]:
plot_X_similar_vec(X, similar_indices,start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.8, 0.05)

In [ ]:
plot_X_similar_vec(X, similar_indices,start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

### Network 2 rho=0.8, alpha = np.linspace(0, 2, 30)[15]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2,0.8, np.linspace(0, 2, 30)[15], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
plot_weighted_degrees(weighted_degrees(W,N))

In [ ]:
plot_X_similar_vec(X, [5,95],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)

In [ ]:
plot_X_similar_vec(X, similar_indices,start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.8, 0.05)

In [ ]:
plot_X_similar_vec(X, similar_indices,start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

### Network 3 rho=0.8, alpha = np.linspace(0, 2, 30)[16]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2,0.8, np.linspace(0, 2, 30)[16], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
plot_weighted_degrees(weighted_degrees(W,N))

In [ ]:
plot_X_similar_vec(X, [5,95],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)

In [ ]:
plot_X_similar_vec(X, similar_indices,start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.8, 0.05)

In [ ]:
plot_X_similar_vec(X, similar_indices,start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

### Network 4 rho=0.8, alpha = np.linspace(0, 2, 30)[12]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2,0.8, np.linspace(0, 2, 30)[12], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
plot_weighted_degrees(weighted_degrees(W,N))

In [ ]:
plot_X_similar_vec(X, [5,95],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)

In [ ]:
plot_X_similar_vec(X, similar_indices,start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.8, 0.05)

In [ ]:
plot_X_similar_vec(X, similar_indices,start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

### Network 5 rho=0.8, alpha = np.linspace(0, 0.25, 30)[9]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2,0.8, np.linspace(0, 0.25, 30)[7], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
plot_weighted_degrees(weighted_degrees(W,N))

In [ ]:
plot_X_similar_vec(X, [5,95],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=False)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)

In [ ]:
plot_X_similar_vec(X, similar_indices,start_time=1000, end_time=1200, legend=False, W=W,N=N, connect_dots=False)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.8, 0.05)

In [ ]:
plot_X_similar_vec(X, similar_indices,start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=False)

#### Network 6 rho=0.8, alpha=np.linspace(0.18,0.23)[19]

In [ ]:
np.linspace(0.18, 0.23, 30)[19]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2,0.8, np.linspace(0.18, 0.23, 30)[19], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
plot_X_similar_vec(X, [5,50,95,85,97, 38,46],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

In [ ]:
plot_X_similar_vec(X, [85, 97],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=True)

In [ ]:
plot_X_similar_vec(X, [38, 46],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=False)

#### Network 7 rho=0.8, alpha=np.linspace(0.18,0.23)[18]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2,0.8, np.linspace(0.18, 0.23, 30)[21], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
plot_X_similar_vec(X, [5,50,95, 85,97, 38,46],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=False)

In [ ]:
plot_X_similar_vec(X, [85, 97],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=False)

In [ ]:
plot_X_similar_vec(X, [38, 46],start_time=1000, end_time=1200, legend=True, W=W,N=N, connect_dots=False)

## F=F_mod, conn="Electrical"

In [ ]:
F=F_mod
conn = "Electrical"

### Network 1 rho=0.8, alpha=0.196

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 0.8, 0.196, 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 95, 0.01)
plot_X_similar_vec(X, similar_indices)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 90, 0.01)
plot_X_similar_vec(X, similar_indices)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 10, 0.01)
plot_X_similar_vec(X, similar_indices)

### Network 2 rho=1.3, alpha=0.196

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 1.3, 0.196, 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 95, 0.01)
plot_X_similar_vec(X, similar_indices)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 95, 0.01)
plot_X_similar_vec(X, similar_indices, start_time=1000, end_time=2000)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 90, 0.01)
plot_X_similar_vec(X, similar_indices)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 90, 0.01)
plot_X_similar_vec(X, similar_indices, start_time=1000, end_time=2000)

### Network 3 rho=0.8, alpha=0.241

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 0.8, 0.241, 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 95, 0.01)
plot_X_similar_vec(X, similar_indices)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 90, 0.01)
plot_X_similar_vec(X, similar_indices)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 90, 0.01)
plot_X_similar_vec(X, similar_indices, start_time=1000, end_time=2000)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 10, 0.01)
plot_X_similar_vec(X, similar_indices)

### Network 4 rho=0.8, alpha=np.linspace(0, 0.25, 15)

In [ ]:
alphas = np.linspace(0, 0.25, 15)

In [ ]:
alphas[11]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 0.8, alphas[11], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
degrees_distribution(W)

In [ ]:
weighted_degrees_distribution(W,N)

In [ ]:
plot_weighted_degrees(weighted_degrees(W,N))

In [ ]:
plot_X_similar_vec(X, [5,95], legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.84, 0.02)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.84, 0.1)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N, start_time=1000, end_time=1100
                   , connect_dots=True)

In [ ]:
plot_X_similar_vec(X, [89], legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.75, 0.01)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, -0.52, 0.005)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, -0.525, 0.005)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, -0.525, 0.005)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N, start_time=1000, end_time=1200, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, -0.53, 0.005)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.425, 0.005)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_X_similar_vec(X, [8], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.425, 0.005)
plot_X_similar_vec(X, [8,7], legend=True, W=W,N=N, start_time=1100, end_time=)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.84, 1)
plot_X_similar_vec(X, similar_indices, legend=False, W=W,N=N)

In [ ]:
ks=weighted_degrees(W,N)

In [ ]:
ks

### Network 5 rho=0.8, alpha=np.linspace(0, 0.25, 30)

In [ ]:
alphas = np.linspace(0,0.25,30)

In [ ]:
alphas[29]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 0.8, alphas[29], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.84, 0.02)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, -0.53, 0.005)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, -0.525, 0.005)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.75, 0.01)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, -0.05, 0.02)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, -0.044, 0.002)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, -0.044, 0.001)
plot_X_similar_vec(X, similar_indices, legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)
plot_X_similar_vec(X, similar_indices, legend=False, W=W,N=N)

### Network 6 rho=0.8, alpha=np.linspace(0

### Network 7 rho=0.8 alpha 0.00542

In [ ]:
alphas = np.linspace(0, 0.01, 60)

In [ ]:
alphas[32]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 0.8, alphas[32], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.84, 0.02)
plot_X_similar_vec(X, [10,38,46], legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)
plot_X_similar_vec(X, similar_indices, legend=False, W=W,N=N )

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)
plot_X_similar_vec(X, [38,46], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 38,46,W,N, 15)

In [ ]:
plot_orbit(X, 38,46,W,N, 20)

In [ ]:
plot_orbit(X, 38,46,W,N, 17)

In [ ]:
plot_orbit(X, 38,46,W,N, 19)

In [ ]:
plot_X_similar_vec(X, [14, 50], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 14,50,W,N, 15)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 1.1, 0.05)
plot_X_similar_vec(X, [6,77], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 6,77,W,N, 17)

In [ ]:
plot_X_similar_vec(X, [1,2,3], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 1,2,W,N, 17)

In [ ]:
plot_X_similar_vec(X, [93,94,95], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 94,93,W,N, 17)

### Network 8 rho=0.8 alpha 0.00338

In [ ]:
alphas = np.linspace(0, 0.01, 60)

In [ ]:
alphas[20]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 0.8, alphas[20], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.84, 0.02)
plot_X_similar_vec(X, [10,38,46], legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)
plot_X_similar_vec(X, similar_indices, legend=False, W=W,N=N )

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)
plot_X_similar_vec(X, [38,46], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 38,46,W,N, 15)

In [ ]:
plot_orbit(X, 38,46,W,N, 20)

In [ ]:
plot_orbit(X, 38,46,W,N, 17)

In [ ]:
plot_orbit(X, 38,46,W,N, 19)

In [ ]:
plot_X_similar_vec(X, [14, 50], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 14,50,W,N, 15)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 1.1, 0.05)
plot_X_similar_vec(X, [6,77], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 6,77,W,N, 17)

In [ ]:
plot_X_similar_vec(X, [1,2,3], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 1,2,W,N, 17)

In [ ]:
plot_X_similar_vec(X, [93,94,95], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 94,93,W,N, 17)

### Network 9 rho=0.8 alpha 0.00169

In [ ]:
alphas = np.linspace(0, 0.01, 60)

In [ ]:
alphas[10]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 0.8, alphas[10], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.84, 0.02)
plot_X_similar_vec(X, [10,38,46], legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)
plot_X_similar_vec(X, similar_indices, legend=False, W=W,N=N )

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)
plot_X_similar_vec(X, [38,46], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 38,46,W,N, 15)

In [ ]:
plot_orbit(X, 38,46,W,N, 20)

In [ ]:
plot_orbit(X, 38,46,W,N, 17)

In [ ]:
plot_orbit(X, 38,46,W,N, 19)

In [ ]:
plot_X_similar_vec(X, [14, 50], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 14,50,W,N, 15)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 1.1, 0.05)
plot_X_similar_vec(X, [6,77], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 6,77,W,N, 17)

In [ ]:
plot_X_similar_vec(X, [1,2,3], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 1,2,W,N, 17)

In [ ]:
plot_X_similar_vec(X, [93,94,95], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_orbit(X, 94,93,W,N, 17)

### Network 10 rho=0.8 alpha 0.00011

In [ ]:
alphas = np.linspace(0, 0.01, 60)

In [ ]:
alphas[1]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 0.8, alphas[1], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.84, 0.02)
plot_X_similar_vec(X, [10,38,46], legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)
plot_X_similar_vec(X, similar_indices, legend=False, W=W,N=N )

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)
plot_X_similar_vec(X, [38,46], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_X_similar_vec(X, [14, 50], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 1.1, 0.05)
plot_X_similar_vec(X, [6,77], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_X_similar_vec(X, [1,2,3], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_X_similar_vec(X, [93,94,95], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

### Network 10 rho=0.8 alpha 0.0002

In [ ]:
alphas = np.linspace(0, 0.001, 60)

In [ ]:
alphas[1]

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 0.8, alphas[1], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0.84, 0.02)
plot_X_similar_vec(X, [10,38,46], legend=True, W=W,N=N)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)
plot_X_similar_vec(X, similar_indices, legend=False, W=W,N=N )

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 0, 5)
plot_X_similar_vec(X, [38,46], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_X_similar_vec(X, [14, 50], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
similar_indices, similar_degrees = find_similar_weighted_degrees(W, N, 1.1, 0.05)
plot_X_similar_vec(X, [6,77], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_X_similar_vec(X, [1,2,3], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

In [ ]:
plot_X_similar_vec(X, [93,94,95], legend=True, W=W,N=N, start_time=1000, end_time=1100, connect_dots=True)

# Orbits

In [ ]:
F=F_mod
conn = "Electrical"

In [ ]:
alphas = np.linspace(0, 0.001, 60)

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 0.8, alphas[1], 10000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate

# Let's assume `states` is a matrix where each row corresponds to the state time series of a neuron

def phase_space_reconstruct(time_series, delay, dimension):
    # The length of the time series that can be used for the reconstruction
    usable_length = len(time_series) - (dimension - 1) * delay
    # Reconstructs the phase space using delay embedding
    reconstructed = np.array([
        time_series[i: i + usable_length]
        for i in range(0, delay * (dimension - 1) + 1, delay)
    ])
    return reconstructed.T

def find_time_lag(ts1, ts2, max_lag):
    # Finds the time lag that maximizes the cross-correlation between two time series
    ts1 = ts1.tolist()
    ts2 = ts2.tolist()
    correlation = correlate(ts1, ts2, mode='full', method='auto')
    lag = np.argmax(correlation) - (len(ts1) - 1)
    return lag if abs(lag) <= max_lag else 0  # If lag is too large, it may not be meaningful

# Parameters for phase space reconstruction
delay = 10  # The delay used in the reconstruction
max_lag = 1000  # Maximum allowed lag


# Reconstruct the phase space for two neurons
neuron1 = phase_space_reconstruct(X[38,:], delay, 2)
neuron2 = phase_space_reconstruct(X[46,:], delay, 2)

# Find the time lag between neuron1 and neuron2
lag = find_time_lag(X[38,:], X[46,:], max_lag)

# Align the neurons by shifting
if lag > 0:
    aligned_neuron1 = neuron1[:-lag]
    aligned_neuron2 = neuron2[lag:]
elif lag < 0:
    aligned_neuron1 = neuron1[-lag:]
    aligned_neuron2 = neuron2[:lag]
else:
    aligned_neuron1 = neuron1
    aligned_neuron2 = neuron2



# Plot the orbits in the reconstructed phase space
plt.plot(aligned_neuron1[:,0], aligned_neuron1[:,1], label='Neuron 1')
plt.plot(aligned_neuron2[:,0], aligned_neuron2[:,1], label='Neuron 2')
plt.xlabel('x(t)')
plt.ylabel('x(t - delay)')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate

# Let's assume `states` is a matrix where each row corresponds to the state time series of a neuron

def phase_space_reconstruct(time_series, delay, dimension):
    # The length of the time series that can be used for the reconstruction
    usable_length = len(time_series) - (dimension - 1) * delay
    # Reconstructs the phase space using delay embedding
    reconstructed = np.array([
        time_series[i: i + usable_length]
        for i in range(0, delay * (dimension - 1) + 1, delay)
    ])
    return reconstructed.T

def find_time_lag(ts1, ts2, max_lag):
    # Finds the time lag that maximizes the cross-correlation between two time series
    ts1 = ts1.tolist()
    ts2 = ts2.tolist()
    correlation = correlate(ts1, ts2, mode='full', method='auto')
    lag = np.argmax(correlation) - (len(ts1) - 1)
    return lag if abs(lag) <= max_lag else 0  # If lag is too large, it may not be meaningful

# Parameters for phase space reconstruction
delay = 50  # The delay used in the reconstruction
max_lag = 1000  # Maximum allowed lag


# Reconstruct the phase space for two neurons
neuron1 = phase_space_reconstruct(X[1,:], delay, 2)
neuron2 = phase_space_reconstruct(X[2,:], delay, 2)

# Find the time lag between neuron1 and neuron2
lag = find_time_lag(X[1,:], X[2,:], max_lag)

# Align the neurons by shifting
if lag > 0:
    aligned_neuron1 = neuron1[:-lag]
    aligned_neuron2 = neuron2[lag:]
elif lag < 0:
    aligned_neuron1 = neuron1[-lag:]
    aligned_neuron2 = neuron2[:lag]
else:
    aligned_neuron1 = neuron1
    aligned_neuron2 = neuron2



# Plot the orbits in the reconstructed phase space
plt.plot(aligned_neuron1[:,0], aligned_neuron1[:,1], label='Neuron 1',alpha=0.3, linewidth=0.2)
plt.plot(aligned_neuron2[:,0], aligned_neuron2[:,1], label='Neuron 2',alpha=0.3, linewidth=0.2)
plt.xlabel('x(t)')
plt.ylabel('x(t - delay)')
plt.legend()
plt.show()


## orbit T=2000

In [ ]:
X, N, epsilon, rho, T, alpha, W = compute_state_matrix(F, conn,
                                                                100, 0.2, 0.8, alphas[1], 2000,   # N, epsilon, rho, alpha
                                                                Lapl=True, ex_time=False)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate

# Let's assume `states` is a matrix where each row corresponds to the state time series of a neuron

def phase_space_reconstruct(time_series, delay, dimension):
    # The length of the time series that can be used for the reconstruction
    usable_length = len(time_series) - (dimension - 1) * delay
    # Reconstructs the phase space using delay embedding
    reconstructed = np.array([
        time_series[i: i + usable_length]
        for i in range(0, delay * (dimension - 1) + 1, delay)
    ])
    return reconstructed.T

def find_time_lag(ts1, ts2, max_lag):
    # Finds the time lag that maximizes the cross-correlation between two time series
    ts1 = ts1.tolist()
    ts2 = ts2.tolist()
    correlation = correlate(ts1, ts2, mode='full', method='auto')
    lag = np.argmax(correlation) - (len(ts1) - 1)
    return lag if abs(lag) <= max_lag else 0  # If lag is too large, it may not be meaningful

# Parameters for phase space reconstruction
delay = 15  # The delay used in the reconstruction
max_lag = 1000  # Maximum allowed lag


# Reconstruct the phase space for two neurons
neuron1 = phase_space_reconstruct(X[1,:], delay, 2)
neuron2 = phase_space_reconstruct(X[2,:], delay, 2)

# Find the time lag between neuron1 and neuron2
lag = find_time_lag(X[1,:], X[2,:], max_lag)

# Align the neurons by shifting
if lag > 0:
    aligned_neuron1 = neuron1[:-lag]
    aligned_neuron2 = neuron2[lag:]
elif lag < 0:
    aligned_neuron1 = neuron1[-lag:]
    aligned_neuron2 = neuron2[:lag]
else:
    aligned_neuron1 = neuron1
    aligned_neuron2 = neuron2



# Plot the orbits in the reconstructed phase space
plt.plot(aligned_neuron1[:,0], aligned_neuron1[:,1], label='Neuron 1',alpha=0.3, linewidth=0.1)
plt.plot(aligned_neuron2[:,0], aligned_neuron2[:,1], label='Neuron 2',alpha=0.3, linewidth=0.2)
plt.xlabel('x(t)')
plt.ylabel('x(t - delay)')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Let's assume `states` is a matrix where each row corresponds to the state time series of a neuron

def phase_space_reconstruct(time_series, delay, dimension):
    # The length of the time series that can be used for the reconstruction
    usable_length = len(time_series) - (dimension - 1) * delay
    # Reconstructs the phase space using delay embedding
    reconstructed = np.array([
        time_series[i: i + usable_length]
        for i in range(0, delay * (dimension - 1) + 1, delay)
    ])
    return reconstructed.T

def find_time_lag(ts1, ts2, max_lag):
    # Finds the time lag that maximizes the cross-correlation between two time series
    ts1 = ts1.tolist()
    ts2 = ts2.tolist()
    correlation = correlate(ts1, ts2, mode='full', method='auto')
    lag = np.argmax(correlation) - (len(ts1) - 1)
    return lag if abs(lag) <= max_lag else 0  # If lag is too large, it may not be meaningful

# Parameters for phase space reconstruction
delay = 15  # The delay used in the reconstruction
max_lag = 1000  # Maximum allowed lag


# Reconstruct the phase space for two neurons
neuron1 = phase_space_reconstruct(X[38,:], delay, 2)
neuron2 = phase_space_reconstruct(X[46,:], delay, 2)

# Find the time lag between neuron1 and neuron2
lag = find_time_lag(X[1,:], X[2,:], max_lag)

# Align the neurons by shifting
if lag > 0:
    aligned_neuron1 = neuron1[:-lag]
    aligned_neuron2 = neuron2[lag:]
elif lag < 0:
    aligned_neuron1 = neuron1[-lag:]
    aligned_neuron2 = neuron2[:lag]
else:
    aligned_neuron1 = neuron1
    aligned_neuron2 = neuron2




# Plot the orbits in the reconstructed phase space
plt.figure(figsize=(10, 6))
plt.plot(aligned_neuron1[:, 0], aligned_neuron1[:, 1], label='Neuron 1', alpha=1, linewidth=0.1, color="red")
plt.plot(aligned_neuron2[:, 0], aligned_neuron2[:, 1], label='Neuron 2', alpha=1, linewidth=0.1, color="blue")
plt.xlabel('x(t)')
plt.ylabel('x(t - delay)')
plt.legend()

# Creating an inset (zoomed-in area)
ax = plt.gca()  # Get the current axes
axins = inset_axes(ax, width="50%", height="50%", loc='upper right')  # Specify the position and size of the inset

# Plot the same data on the inset, focusing on the specified range
axins.plot(aligned_neuron1[:, 0], aligned_neuron1[:, 1], alpha=1, linewidth=0.1, color="red")
axins.plot(aligned_neuron2[:, 0], aligned_neuron2[:, 1], alpha=1, linewidth=0.1, color="blue")

# Set the x and y axis limits to zoom in to the specific range (0.1 to 0.3 for x(t))
axins.set_xlim(0.2, 0.4)
axins.set_ylim(0, 0.1)  # Set appropriate limits for y-axis based on your data

# Optional: Add grid, labels, or customize further as needed
axins.grid(True)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Let's assume `states` is a matrix where each row corresponds to the state time series of a neuron

def phase_space_reconstruct(time_series, delay, dimension):
    # The length of the time series that can be used for the reconstruction
    usable_length = len(time_series) - (dimension - 1) * delay
    # Reconstructs the phase space using delay embedding
    reconstructed = np.array([
        time_series[i: i + usable_length]
        for i in range(0, delay * (dimension - 1) + 1, delay)
    ])
    return reconstructed.T

def find_time_lag(ts1, ts2, max_lag):
    # Finds the time lag that maximizes the cross-correlation between two time series
    ts1 = ts1.tolist()
    ts2 = ts2.tolist()
    correlation = correlate(ts1, ts2, mode='full', method='auto')
    lag = np.argmax(correlation) - (len(ts1) - 1)
    return lag if abs(lag) <= max_lag else 0  # If lag is too large, it may not be meaningful

# Parameters for phase space reconstruction
delay = 15  # The delay used in the reconstruction
max_lag = 1000  # Maximum allowed lag


# Reconstruct the phase space for two neurons
neuron1 = phase_space_reconstruct(X[38,:], delay, 2)
neuron2 = phase_space_reconstruct(X[46,:], delay, 2)

# Find the time lag between neuron1 and neuron2
lag = find_time_lag(X[38,:], X[46,:], max_lag)
print(lag)
# Align the neurons by shifting
if lag > 0:
    aligned_neuron1 = neuron1[:-lag]
    aligned_neuron2 = neuron2[lag:]
elif lag < 0:
    aligned_neuron1 = neuron1[-lag:]
    aligned_neuron2 = neuron2[:lag]
else:
    aligned_neuron1 = neuron1
    aligned_neuron2 = neuron2




# Plot the orbits in the reconstructed phase space
plt.figure(figsize=(10, 6))
plt.plot(aligned_neuron1[:, 0], aligned_neuron1[:, 1], label='Neuron 1', alpha=1, linewidth=0.1, color="red")
plt.plot(aligned_neuron2[:, 0], aligned_neuron2[:, 1], label='Neuron 2', alpha=1, linewidth=0.1, color="blue")
plt.xlabel('x(t)')
plt.ylabel('x(t - delay)')
plt.legend()

# Creating an inset (zoomed-in area)
ax = plt.gca()  # Get the current axes
axins = inset_axes(ax, width="50%", height="50%", loc='upper right')  # Specify the position and size of the inset

# Plot the same data on the inset, focusing on the specified range
axins.plot(aligned_neuron1[:, 0], aligned_neuron1[:, 1], alpha=1, linewidth=0.1, color="red")
axins.plot(aligned_neuron2[:, 0], aligned_neuron2[:, 1], alpha=1, linewidth=0.1, color="blue")

# Set the x and y axis limits to zoom in to the specific range (0.1 to 0.3 for x(t))
axins.set_xlim(0.25, 0.3)
axins.set_ylim(0.02, 0.06)  # Set appropriate limits for y-axis based on your data

# Optional: Add grid, labels, or customize further as needed
axins.grid(True)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Let's assume `states` is a matrix where each row corresponds to the state time series of a neuron

def phase_space_reconstruct(time_series, delay, dimension):
    # The length of the time series that can be used for the reconstruction
    usable_length = len(time_series) - (dimension - 1) * delay
    # Reconstructs the phase space using delay embedding
    reconstructed = np.array([
        time_series[i: i + usable_length]
        for i in range(0, delay * (dimension - 1) + 1, delay)
    ])
    return reconstructed.T

def find_time_lag(ts1, ts2, max_lag):
    # Finds the time lag that maximizes the cross-correlation between two time series
    ts1 = ts1.tolist()
    ts2 = ts2.tolist()
    correlation = correlate(ts1, ts2, mode='full', method='auto')
    lag = np.argmax(correlation) - (len(ts1) - 1)
    return lag if abs(lag) <= max_lag else 0  # If lag is too large, it may not be meaningful

# Parameters for phase space reconstruction
delay = 15  # The delay used in the reconstruction
max_lag = 1000  # Maximum allowed lag


# Reconstruct the phase space for two neurons
neuron1 = phase_space_reconstruct(X[38,:], delay, 2)
neuron2 = phase_space_reconstruct(X[46,:], delay, 2)

# Find the time lag between neuron1 and neuron2
lag = find_time_lag(X[38,:], X[46,:], max_lag)
print(lag)
# Align the neurons by shifting
if lag > 0:
    aligned_neuron1 = neuron1[:-lag]
    aligned_neuron2 = neuron2[lag:]
elif lag < 0:
    aligned_neuron1 = neuron1[-lag:]
    aligned_neuron2 = neuron2[:lag]
else:
    aligned_neuron1 = neuron1
    aligned_neuron2 = neuron2




# Plot the orbits in the reconstructed phase space
plt.figure(figsize=(10, 6))
plt.plot(aligned_neuron1[:, 0], aligned_neuron1[:, 1], label='Neuron 1', alpha=1, linewidth=0.1, color="red")
plt.plot(aligned_neuron2[:, 0], aligned_neuron2[:, 1], label='Neuron 2', alpha=1, linewidth=0.1, color="blue")
plt.xlabel('x(t)')
plt.ylabel('x(t - delay)')
plt.legend()

# Creating an inset (zoomed-in area)
ax = plt.gca()  # Get the current axes
axins = inset_axes(ax, width="50%", height="50%", loc='upper right')  # Specify the position and size of the inset

# Plot the same data on the inset, focusing on the specified range
axins.plot(aligned_neuron1[:, 0], aligned_neuron1[:, 1], alpha=1, linewidth=0.1, color="red")
axins.plot(aligned_neuron2[:, 0], aligned_neuron2[:, 1], alpha=1, linewidth=0.1, color="blue")

# Set the x and y axis limits to zoom in to the specific range (0.1 to 0.3 for x(t))
axins.set_xlim(0.2, 0.4)
axins.set_ylim(0.02, 0.1)  # Set appropriate limits for y-axis based on your data


plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Let's assume `states` is a matrix where each row corresponds to the state time series of a neuron

def phase_space_reconstruct(time_series, delay, dimension):
    # The length of the time series that can be used for the reconstruction
    usable_length = len(time_series) - (dimension - 1) * delay
    # Reconstructs the phase space using delay embedding
    reconstructed = np.array([
        time_series[i: i + usable_length]
        for i in range(0, delay * (dimension - 1) + 1, delay)
    ])
    return reconstructed.T

def find_time_lag(ts1, ts2, max_lag):
    # Finds the time lag that maximizes the cross-correlation between two time series
    ts1 = ts1.tolist()
    ts2 = ts2.tolist()
    correlation = correlate(ts1, ts2, mode='full', method='auto')
    lag = np.argmax(correlation) - (len(ts1) - 1)
    return lag if abs(lag) <= max_lag else 0  # If lag is too large, it may not be meaningful

def plot_orbit(X, n1, n2, W,N, delay=15):
  # Parameters for phase space reconstruction

  max_lag = 1000  # Maximum allowed lag
  ks = weighted_degrees_unsorted(W,N)
  nonzero_counts = np.count_nonzero(W, axis=1)

  # Reconstruct the phase space for two neurons
  neuron1 = phase_space_reconstruct(X[n1,:], delay, 2)
  neuron2 = phase_space_reconstruct(X[n2,:], delay, 2)

  # Find the time lag between neuron1 and neuron2
  lag = find_time_lag(X[n1,:], X[n2,:], max_lag)
  print(lag)
  # Align the neurons by shifting
  if lag > 0:
      aligned_neuron1 = neuron1[:-lag]
      aligned_neuron2 = neuron2[lag:]
  elif lag < 0:
      aligned_neuron1 = neuron1[-lag:]
      aligned_neuron2 = neuron2[:lag]
  else:
      aligned_neuron1 = neuron1
      aligned_neuron2 = neuron2




  # Plot the orbits in the reconstructed phase space
  plt.figure(figsize=(10, 6))
  plt.plot(aligned_neuron1[:, 0], aligned_neuron1[:, 1],label=f"Neuron {n1} (k={ks[n1]:.3f})", alpha=1, linewidth=0.3, color="red")
  plt.plot(aligned_neuron2[:, 0], aligned_neuron2[:, 1], label=f"Neuron {n2} (k={ks[n2]:.3f})", alpha=1, linewidth=0.3, color="blue")
  plt.xlabel('s(t)', fontsize=15)
  plt.ylabel('s(t - delay)', fontsize=15)
  plt.legend(loc = "upper left", fontsize=15)
  plt.tick_params(axis='both', labelsize=15)

  # Creating an inset (zoomed-in area)
  ax = plt.gca()  # Get the current axes
  axins = inset_axes(ax, width="50%", height="50%", loc='upper right')  # Specify the position and size of the inset

  # Plot the same data on the inset, focusing on the specified range
  axins.plot(aligned_neuron1[:, 0], aligned_neuron1[:, 1], alpha=1, linewidth=0.3, color="red")
  axins.plot(aligned_neuron2[:, 0], aligned_neuron2[:, 1], alpha=1, linewidth=0.3, color="blue")

  # Set the x and y axis limits to zoom in to the specific range (0.1 to 0.3 for x(t))
  axins.set_xlim(0.2, 0.4)
  axins.set_ylim(0.02, 0.1)  # Set appropriate limits for y-axis based on your data


  plt.show()

In [ ]:
plot_orbit(X, 38,46,W,N)

In [ ]:
plot_orbit(X, 1,2,W,N)